<a href="https://colab.research.google.com/github/cristianscu99-VIU/AOP-ACT-1/blob/main/SEMINARIO/Feedback_Proyecto_AOP_Problema_2_Cristian_de_la_Santa_Cruz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: **Cristian de la Santa Cruz Urbano**   <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO<br>
Problema:
> 1. Sesiones de doblaje <br>
>2. **Organizar los horarios de partidos de La Liga**<br>
>3. Combinar cifras y operaciones

**Descripción del problema:**

- Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.

- Los horarios disponibles se conocen a priori y son los siguientes:


In [ ]:
# @title
  Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                 'D20' : 1, 'L20' : 0.4}
  listado_horarios = list(Ponderacion.keys())
  listado_horarios

['V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20', 'L20']

- En primer lugar se clasifican los equipos en tres categorías según el numero de seguidores( que tiene relación directa con la audiencia). Hay 3 equiposen la categoría A,11 equiposde categoría By 6 equiposde categoría C.

- Se conoce estadísticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos)

In [ ]:
# @title
  import pandas as pd
  audiencia_base = {
      'A' : {'A' : 2},
      'B' : {'A' : 1.3, 'B' : 0.9},
      'C' : {'A' : 1, 'B' : 0.75, 'C' : 0.47}
  }
  df = pd.DataFrame(audiencia_base)
  display(df)

,A,B,C
A,2.0,1.3,1.00
B,NaN,0.9,0.75
C,NaN,NaN,0.47


- Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes de la siguiente tabla
- Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes

In [ ]:
# @title
Ponderacion = {'Viernes' : {'20h' : 0.4},
             'Sábado' : {'12h' : 0.55, '16h' : 0.7, '18h' : 0.8, '20h' : 1},
             'Domingo' : {'12h' : 0.45, '16h' : 0.75, '18h' : 0.85, '20h' : 1},
             'Lunes' : {'20h' : 0.4}
             }
Ponderacion = pd.DataFrame(Ponderacion).sort_index(ascending=True)
display(Ponderacion)

,Viernes,Sábado,Domingo,Lunes
12h,NaN,0.55,0.45,NaN
16h,NaN,0.70,0.75,NaN
18h,NaN,0.80,0.85,NaN
20h,0.4,1.00,1.00,0.4


- Es posible la coincidencia de horarios pero en este caso la audiencia de cada partido se verá afectada y se estima que se reduce en porcentaje según la siguiente tabla dependiendo del número de coincidencias:

In [ ]:
# @title
Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}

df_Coincidencias = pd.DataFrame(Coincidencias)
df_Coincidencias.columns.name='Coincidencias'
display(df_Coincidencias)

Coincidencias,-%
0,0.00
1,0.25
2,0.45
3,0.60
4,0.70
5,0.75
6,0.78
7,0.80
8,0.80


- Los cálculos asociados a una jornada de ejemplo se realizan según se muestra en la siguiente tabla:

In [ ]:
# @title
import random
import numpy as np
import pandas as pd

dict_equipos = {'Celta' : 'B', 'Real Madrid' : 'A', 'Valencia' : 'B', 'R.Sociedad' : 'A', 'Mallorca' : 'C', 'Eibar' : 'C',
           'Athletic' : 'B', 'Barcelona' : 'A', 'Leganés' : 'C', 'Osasuna' : 'C', 'Villareal' : 'B', 'Granada' : 'C',
           'Alavés' : 'B', 'Levante' : 'B', 'Espayol' : 'B', 'Sevilla' : 'B', 'Betis' : 'B', 'Valladolid' : 'C', 'Atlético' : 'B',
           'Getafe' : 'B'
           }

def Correcion_por_coincidencia(lista_horario_partido): # devuelve una lista con las diferentes correciones en base a la coincidencia
  Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
  lista_correciones = []

  for hora in lista_horario_partido:
    contador = lista_horario_partido.count(hora) - 1
    lista_correciones.append(Coincidencias['-%'][contador])
  return lista_correciones

def generar_partidos (dict_equipos):
  "Función que devuelve los partidos que se van a jugar"
  lista_partidos = [] # lista que guarda los distintos partidos de una jornada
  lista_categorias = [] # lista que guarda las distintas categorias de los partidos de una jornada
  lista_bases = [] # lista con las bases de audiencia de cada partido en funcion de la categoría
  lista_equipos = list(dict_equipos.keys()) #lista con todos los equipos
  lista_equipos_disponibles = lista_equipos[:]

  lista_partidos = [
      'Celta - Real Madrid',
      'Valencia - R.Sociedad',
      'Mallorca - Eibar',
      'Athletic - Barcelona',
      'Leganés - Osasuna',
      'Villareal - Granada',
      'Alavés - Levante',
      'Espayol - Sevilla',
      'Betis - Valladolid',
      'Atlético - Getafe',
  ]

  lista_categorias = [
      'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B', 'B-C', 'B-B'
  ]

  lista_bases = [
      1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75, 0.9
  ]


  Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                 'D20' : 1, 'L20' : 0.4}

  listado_horarios = list(Ponderacion.keys())

  lista_horario_partido = [
      'V20', 'S12', 'S16', 'S18', 'S20', 'D16', 'D16', 'D18', 'D20', 'L20'
  ]

  lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]

  return lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones

lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones = generar_partidos(dict_equipos)
lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)

dict_jornada = {
    'Partido' : lista_partidos,
    'Categorías' : lista_categorias,
    'Horario' : lista_horario_partido,
    'Base(Mill.)' : lista_bases,
    'Ponderación' : lista_ponderaciones,
    'Base*Ponderación' : lista_base_ponderacion,
    'Correción Coincidencia' : lista_correciones_corregida,
}

df_jornada = pd.DataFrame(dict_jornada)
display(df_jornada)
Total = np.sum(lista_correciones_corregida)
print('\n Total: ',Total)

,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,V20,1.30,0.40,0.5200,0.520000
1,Valencia - R.Sociedad,B-A,S12,1.30,0.55,0.7150,0.715000
2,Mallorca - Eibar,C-C,S16,0.47,0.70,0.3290,0.329000
3,Athletic - Barcelona,B-A,S18,1.30,0.80,1.0400,1.040000
4,Leganés - Osasuna,C-C,S20,0.47,1.00,0.4700,0.470000
5,Villareal - Granada,B-C,D16,0.75,0.75,0.5625,0.421875
6,Alavés - Levante,B-B,D16,0.90,0.75,0.6750,0.506250
7,Espayol - Sevilla,B-B,D18,0.90,0.85,0.7650,0.765000
8,Betis - Valladolid,B-C,D20,0.75,1.00,0.7500,0.750000
9,Atlético - Getafe,B-B,L20,0.90,0.40,0.3600,0.360000



 Total:  5.877125


# Preguntas

**(*) ¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>**



**¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.**

Respuesta
- a) Cómo la cantidad de partidos es P = 10 y la cantidad de horarios distintos es H = 10, tenemos que H^P = 10^10 = 1*10^10, es decir, 10 mil millones de combinaciones posibles.

- b) Sabiendo que las restricciones son que obligatoriamente debe haber al menos un partido el Viernes y otro partido el Lunes y se corresponden con el primer y último horario respectivamente. Tenemos que la cantidad de soluciones a calcular sería alrededor de 4 mil 100 millones de combinaciones posibles.

In [ ]:
H = 10 # Cantidad de horarios distintos. H = len(listado_horarios)
P = 10 # Cantidad de partidos distintos en una jornada. P = len(lista_partidos)
print(f'a) Combinatoria sin restricciones: {H**P}')
print(f'\nb) Combinatoria con restricciones:{H**P} - {(H-1)**P} - {(H-1)**P} + {(H-2)**P} = {H**P - (H-1)**P - (H-1)**P + (H-2)**P}')

a) Combinatoria sin restricciones: 10000000000

b) Combinatoria con restricciones:10000000000 - 3486784401 - 3486784401 + 1073741824 = 4100173022


Explicación:

Para calcular el número de posibles soluciones es necesario calcular a través de combinatoria la cantidad de situaciones diferentes a evaluar.
- **Sin restrincciones:**
Suponiendo que tengamos P partidos en 1 jornada, y H horarios diferentes posibles para poder jugar cada partido. Tendríamos que ir evaluando el resultado de la audiencia de esa jornada para cada partido. Cada partido tiene H horarios diferentes en los que poder jugar.
Supongamos el caso en que P = 4 y H = 3

In [ ]:
# @title
from itertools import product

P = 4
H = 3

# Generar todas las combinaciones de P números del 0 a H-1
matriz = list(product(range(H), repeat=P))

print(f'La cantidad de combinaciones diferentes de horarios entre partidos es {len(matriz)}'
      f'\nque es lo mismo que la combinatoria H^(P), es decir {H}^{P} = {H**P}. \n\n Combinatoria:')
matriz

# Index - PARTIDO
# Value - HORARIO (0 es el primer horario)

La cantidad de combinaciones diferentes de horarios entre partidos es 81
que es lo mismo que la combinatoria H^(P), es decir 3^4 = 81. 

 Combinatoria:


[(0, 0, 0, 0),
 (0, 0, 0, 1),
 (0, 0, 0, 2),
 (0, 0, 1, 0),
 (0, 0, 1, 1),
 (0, 0, 1, 2),
 (0, 0, 2, 0),
 (0, 0, 2, 1),
 (0, 0, 2, 2),
 (0, 1, 0, 0),
 (0, 1, 0, 1),
 (0, 1, 0, 2),
 (0, 1, 1, 0),
 (0, 1, 1, 1),
 (0, 1, 1, 2),
 (0, 1, 2, 0),
 (0, 1, 2, 1),
 (0, 1, 2, 2),
 (0, 2, 0, 0),
 (0, 2, 0, 1),
 (0, 2, 0, 2),
 (0, 2, 1, 0),
 (0, 2, 1, 1),
 (0, 2, 1, 2),
 (0, 2, 2, 0),
 (0, 2, 2, 1),
 (0, 2, 2, 2),
 (1, 0, 0, 0),
 (1, 0, 0, 1),
 (1, 0, 0, 2),
 (1, 0, 1, 0),
 (1, 0, 1, 1),
 (1, 0, 1, 2),
 (1, 0, 2, 0),
 (1, 0, 2, 1),
 (1, 0, 2, 2),
 (1, 1, 0, 0),
 (1, 1, 0, 1),
 (1, 1, 0, 2),
 (1, 1, 1, 0),
 (1, 1, 1, 1),
 (1, 1, 1, 2),
 (1, 1, 2, 0),
 (1, 1, 2, 1),
 (1, 1, 2, 2),
 (1, 2, 0, 0),
 (1, 2, 0, 1),
 (1, 2, 0, 2),
 (1, 2, 1, 0),
 (1, 2, 1, 1),
 (1, 2, 1, 2),
 (1, 2, 2, 0),
 (1, 2, 2, 1),
 (1, 2, 2, 2),
 (2, 0, 0, 0),
 (2, 0, 0, 1),
 (2, 0, 0, 2),
 (2, 0, 1, 0),
 (2, 0, 1, 1),
 (2, 0, 1, 2),
 (2, 0, 2, 0),
 (2, 0, 2, 1),
 (2, 0, 2, 2),
 (2, 1, 0, 0),
 (2, 1, 0, 1),
 (2, 1, 0, 2),
 (2, 1, 1,

- **Con restricciones:** Teniendo en cuenta todas las restricciones, tendríamos que analizar todas aquellas posibles soluciones que contengan como partida al menos 1 partido en el primer horario (Viernes) y un partido en el último horario (Lunes). Suponemos P=4 y H=3.

In [ ]:
from itertools import product

P = 5
H = 4

# Generar todas las combinaciones de P números del 0 a H-1
matriz = list(product(range(H), repeat=P))
matriz_restringida = [tuple for tuple in matriz if (0 in tuple and H-1 in tuple)]

print(f'La cantidad de combinaciones diferentes de horarios entre partidos con restricciones es {len(matriz_restringida)}.'
      f'\nQue es lo mismo que la combinatoria H^(P) - A - B + AintersecciónB,'
      f'\nsiendo A el conjunto formado por las filas que no contienen el primer horario'
      f'\ny B el conjunto formado por las filas que no contienen el último horario, es decir:'
      f'\nH = {[i for i in range(H)]}, A = {H-1}^P (A es subconjunto de H que no tiene 0) y de la misma forma,'
      f'\nB = {H-1}^P (B es subconjunto de H que no tiene {H-1}) y'
      f'\nAintersecciónB = {H-2}^P (AintersecciónB es subconjunto de H que no tiene 0 ni {H-1})'
      f'\nPor tanto: H^(P) - A - B + AintersecciónB = {H**P}-{(H-1)**P}-{(H-1)**P}+{(H-2)**P} = {H**P-2*(H-1)**P+(H-2)**P} \n\n Combinatoria:')

matriz_restringida

# Index - PARTIDO
# Value - HORARIO (0 es el primer horario)

La cantidad de combinaciones diferentes de horarios entre partidos con restricciones es 570.
Que es lo mismo que la combinatoria H^(P) - A - B + AintersecciónB,
siendo A el conjunto formado por las filas que no contienen el primer horario
y B el conjunto formado por las filas que no contienen el último horario, es decir:
H = [0, 1, 2, 3], A = 3^P (A es subconjunto de H que no tiene 0) y de la misma forma,
B = 3^P (B es subconjunto de H que no tiene 3) y
AintersecciónB = 2^P (AintersecciónB es subconjunto de H que no tiene 0 ni 3)
Por tanto: H^(P) - A - B + AintersecciónB = 1024-243-243+32 = 570 

 Combinatoria:


[(0, 0, 0, 0, 3),
 (0, 0, 0, 1, 3),
 (0, 0, 0, 2, 3),
 (0, 0, 0, 3, 0),
 (0, 0, 0, 3, 1),
 (0, 0, 0, 3, 2),
 (0, 0, 0, 3, 3),
 (0, 0, 1, 0, 3),
 (0, 0, 1, 1, 3),
 (0, 0, 1, 2, 3),
 (0, 0, 1, 3, 0),
 (0, 0, 1, 3, 1),
 (0, 0, 1, 3, 2),
 (0, 0, 1, 3, 3),
 (0, 0, 2, 0, 3),
 (0, 0, 2, 1, 3),
 (0, 0, 2, 2, 3),
 (0, 0, 2, 3, 0),
 (0, 0, 2, 3, 1),
 (0, 0, 2, 3, 2),
 (0, 0, 2, 3, 3),
 (0, 0, 3, 0, 0),
 (0, 0, 3, 0, 1),
 (0, 0, 3, 0, 2),
 (0, 0, 3, 0, 3),
 (0, 0, 3, 1, 0),
 (0, 0, 3, 1, 1),
 (0, 0, 3, 1, 2),
 (0, 0, 3, 1, 3),
 (0, 0, 3, 2, 0),
 (0, 0, 3, 2, 1),
 (0, 0, 3, 2, 2),
 (0, 0, 3, 2, 3),
 (0, 0, 3, 3, 0),
 (0, 0, 3, 3, 1),
 (0, 0, 3, 3, 2),
 (0, 0, 3, 3, 3),
 (0, 1, 0, 0, 3),
 (0, 1, 0, 1, 3),
 (0, 1, 0, 2, 3),
 (0, 1, 0, 3, 0),
 (0, 1, 0, 3, 1),
 (0, 1, 0, 3, 2),
 (0, 1, 0, 3, 3),
 (0, 1, 1, 0, 3),
 (0, 1, 1, 1, 3),
 (0, 1, 1, 2, 3),
 (0, 1, 1, 3, 0),
 (0, 1, 1, 3, 1),
 (0, 1, 1, 3, 2),
 (0, 1, 1, 3, 3),
 (0, 1, 2, 0, 3),
 (0, 1, 2, 1, 3),
 (0, 1, 2, 2, 3),
 (0, 1, 2, 3, 0),
 (0, 1, 2,

In [ ]:
from itertools import product

def combinaciones_validas(P, H):
    for comb in product(range(H), repeat=P):
        if 0 in comb and (H - 1) in comb:
            yield comb

lista = []
gen = combinaciones_validas(9, 9)
for i, fila in enumerate(gen):
  if i == 10000000:
    print(fila)
  if i == 50000000:
    print(fila)
  if i == 9**9-1:
    print(fila)
#     lista.append(fila)
# print(len(lista))

(0, 3, 7, 2, 1, 8, 8, 6, 5)
(2, 5, 0, 5, 2, 8, 8, 4, 1)


**Modelo para el espacio de soluciones**<br>
**(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguméntalo)**

Respuesta

En un primer escenario he propuesto la siguiente estructuras de datos:
Diccionarios : dict_equipos, Coincidencias
Listas : lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones, lista_base_ponderacion,lista_correciones, lista_correciones_corregida
DataFrames : df_jornada

**Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?**

**(*)¿Es un problema de maximización o minimización?**

Respuesta

La función objetivo es el cálculo de la audiencia en base a la organización horaria de los partidos (lista_horario_partido).

Se trata de un problema de maximización de audiencia.

**Diseña un algoritmo para resolver el problema por fuerza bruta**

## Versiones Optimizadas

In [ ]:
# def Correcion_por_coincidencia(lista_horario_partido):
#   Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
#   lista_repeticiones = []

#   unique, counts = np.unique(lista_horario_partido, return_counts=True)
#   dict_valores = dict(zip(unique, counts))

#   for valor in lista_horario_partido:
#     repeticiones = dict_valores[valor] - 1 # restamos 1 porque si está le corresponde una correción de coincidencia de -0% no del -25%
#     lista_repeticiones.append(repeticiones)

#   lista_correciones = np.array(Coincidencias['-%'])[np.array(lista_repeticiones)]
#   return lista_correciones

##REEMPLAZADO POR: (+30% performance) antes teniamos el .count() pero al usar numpy no pude usarlo y por eso se usaba unique, counts. Volvemos a la idea original con Counter de la librería collections.

# Función que devuelve una lista con las diferentes correciones en base a la coincidencia
from collections import Counter
def Correcion_por_coincidencia(lista_horario_partido):
    #Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]} cogemos directamente la lista para no llamar al diccionario
    correcciones = [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]
    contador = Counter(lista_horario_partido)

    # Aplicar la corrección correspondiente a cada valor
    lista_correciones = [correcciones[min(contador[hora] - 1, len(correcciones) - 1)] for hora in lista_horario_partido]

    return np.array(lista_correciones)



In [ ]:
def problema():
  import numpy as np
  import pandas as pd
  from itertools import product
  import time
  from collections import Counter

  # Función que devuelve una lista con las diferentes correciones en base a la coincidencia
  def Correcion_por_coincidencia(lista_horario_partido):
      #Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]} cogemos directamente la lista para no llamar al diccionario
      correcciones = [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]
      contador = Counter(lista_horario_partido)

      # Aplicar la corrección correspondiente a cada valor
      lista_correciones = [correcciones[min(contador[hora] - 1, len(correcciones) - 1)] for hora in lista_horario_partido]

      return np.array(lista_correciones)

  # Función que genera dataframe
  def generar_dataframe(lista_partidos: list, lista_categorias: list, lista_horario_partido:list, lista_bases: list, lista_ponderaciones:list, lista_base_ponderacion:list, lista_correciones_corregida:list ):

    dict_jornada = {
        'Partido' : lista_partidos,
        'Categorías' : lista_categorias,
        'Horario' : lista_horario_partido,
        'Base(Mill.)' : lista_bases,
        'Ponderación' : lista_ponderaciones,
        'Base*Ponderación' : lista_base_ponderacion,
        'Correción Coincidencia' : lista_correciones_corregida,
    }

    df_jornada = pd.DataFrame(dict_jornada)
    return df_jornada

  # Función que genera combinación con restrincciones
  def combinaciones_validas(P, H):
      for comb in product(range(H), repeat=P):
          if 0 in comb and (H - 1) in comb:
              yield comb


  # Condiciones iniciales para un problema fijo (P = PARTIDOS, H = HORARIOS)

  #############################################
  # PROBLEMA P = 4 H = 3
  #############################################

  # lista_partidos = [
  #     'Celta - Real Madrid',
  #     'Valencia - R.Sociedad',
  #     'Mallorca - Eibar',
  #     'Athletic - Barcelona',
  # ]

  # lista_categorias = ['B-A', 'B-A', 'C-C', 'B-A']

  # lista_bases = [1.3, 1.3, 0.47, 1.3]

  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7}

  #############################################
  # PROBLEMA P = 7 H = 7
  #############################################
  lista_partidos = [
        'Celta - Real Madrid',
        'Valencia - R.Sociedad',
        'Mallorca - Eibar',
        'Athletic - Barcelona',
        'Leganés - Osasuna',
        'Villareal - Granada',
        'Alavés - Levante',
    ]

  lista_categorias = [
        'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B'
    ]

  lista_bases = [
        1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9
    ]


  Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                   'S20' : 1, 'D12' : 0.45, 'D16' : 0.75}

  #############################################
    # PROBLEMA P = 8 H = 8
  #############################################


  # lista_partidos = [
  #       'Celta - Real Madrid',
  #       'Valencia - R.Sociedad',
  #       'Mallorca - Eibar',
  #       'Athletic - Barcelona',
  #       'Leganés - Osasuna',
  #       'Villareal - Granada',
  #       'Alavés - Levante',
  #       'Espayol - Sevilla',
  #   ]

  # lista_categorias = [
  #       'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B'
  #   ]

  # lista_bases = [
  #       1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9
  #   ]


  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
  #                  'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85}


  #############################################
  # PROBLEMA P = 9 H = 10
  #############################################

  # lista_partidos = [
  #   'Celta - Real Madrid',
  #   'Valencia - R.Sociedad',
  #   'Mallorca - Eibar',
  #   'Athletic - Barcelona',
  #   'Leganés - Osasuna',
  #   'Villareal - Granada',
  #   'Alavés - Levante',
  #   'Espayol - Sevilla',
  #   'Betis - Valladolid',
  #   ]

  # lista_categorias = [
  #     'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B', 'B-C'
  # ]

  # lista_bases = [
  #     1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75
  # ]


  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
  #                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
  #                 'D20' : 1, 'L20' : 0.4}

  ##############################################
  #############################################
    # PROBLEMA P = 10 H = 10
  #############################################

  # lista_partidos = [
  #   'Celta - Real Madrid',
  #   'Valencia - R.Sociedad',
  #   'Mallorca - Eibar',
  #   'Athletic - Barcelona',
  #   'Leganés - Osasuna',
  #   'Villareal - Granada',
  #   'Alavés - Levante',
  #   'Espayol - Sevilla',
  #   'Betis - Valladolid',
  #   'Atlético - Getafe',
  #   ]

  # lista_categorias = [
  #     'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B', 'B-C', 'B-B'
  # ]

  # lista_bases = [
  #     1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75, 0.9
  # ]


  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
  #                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
  #                 'D20' : 1, 'L20' : 0.4}

  ##############################################

  Total_max = 0
  horario_max_audiencia = []

  listado_horarios = list(Ponderacion.keys())

  P = len(lista_partidos)
  H = len(listado_horarios)
  combinaciones_totales = (H**P - (H-1)**P - (H-1)**P + (H-2)**P)

  # Generador de combinaciones teniendo en cuentra restrincciones
  gen = combinaciones_validas(P, H)

  start_time = time.time()  # Inicia temporizador
  # Cálculo de la audiencia para cada combinación posible
  for i, fila in enumerate(gen):
    fila_array = np.array(fila)
    lista_horario_partido = np.array(listado_horarios)[fila_array] # lista de listas con todos los horarios posibles para la jornada
    lista_ponderaciones = np.array([Ponderacion[h] for h in lista_horario_partido])
    lista_base_ponderacion = np.array(lista_bases)*lista_ponderaciones
    lista_correciones = Correcion_por_coincidencia(lista_horario_partido)
    lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)
    Total = np.sum(lista_correciones_corregida)

    if Total > Total_max:
      Total_max = Total
      #horario_max_audiencia = [] # vaciamos la lista
      horario_max_audiencia = [lista_horario_partido.tolist()]
      print(f'Combinación número: {i}, Máxima Audiencia encontrada: {Total_max:.2f}')
      end_time = time.time()  # Finaliza temporizador
      print(f"Tiempo total de ejecución: {int(end_time - start_time)} segundos")

      if i>1:
        tiempo_estimado_total = (combinaciones_totales*(end_time - start_time)/i)
        horas = int(tiempo_estimado_total//3600)
        minutos = int((tiempo_estimado_total %3600)//60)
        segundos = int(tiempo_estimado_total-horas*3600)%60
        print(f"Tiempo total estimado: {horas} horas, {minutos} minutos y {segundos} segundos")

    elif Total == Total_max:
      horario_max_audiencia.append(lista_horario_partido.tolist())

  # ¿1 Solución o varias?
  if len(horario_max_audiencia) == 1:
    print(f'Solución: El horario con máxima audiencia es: {horario_max_audiencia} y obtiene una audiencia de {Total_max} \n')
  elif len(horario_max_audiencia) > 1:
    print(f'Solución: Los horarios con máxima audiencia son: {len(horario_max_audiencia)} y se obtiene una audiencia de {Total_max} en cualquiera de estas combinaciones \n')

  # Mostramos el DataFrame para los horarios que producen una máxima audiencia en una jornada
  for lista_horario_partido in horario_max_audiencia:
    lista_ponderaciones = np.array([Ponderacion[h] for h in lista_horario_partido])
    lista_base_ponderacion = np.array(lista_bases)*lista_ponderaciones
    lista_correciones = Correcion_por_coincidencia(lista_horario_partido)
    lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)
    df_jornada_max_audiencia = generar_dataframe(lista_partidos, lista_categorias, lista_horario_partido, lista_bases, lista_ponderaciones, lista_base_ponderacion, lista_correciones_corregida)
    display(df_jornada_max_audiencia)
    Total = np.sum(lista_correciones_corregida)
    print(f'La audiencia total esperada es: {Total} M. \n')

In [ ]:
problema()

Combinación número: 0, Máxima Audiencia encontrada: 1.23
Tiempo total de ejecución: 0 segundos
Combinación número: 1, Máxima Audiencia encontrada: 1.67
Tiempo total de ejecución: 0 segundos
Combinación número: 2, Máxima Audiencia encontrada: 1.78
Tiempo total de ejecución: 0 segundos
Tiempo total estimado: 0 horas, 1 minutos y 24 segundos
Combinación número: 3, Máxima Audiencia encontrada: 1.86
Tiempo total de ejecución: 0 segundos
Tiempo total estimado: 0 horas, 1 minutos y 7 segundos
Combinación número: 4, Máxima Audiencia encontrada: 2.01
Tiempo total de ejecución: 0 segundos
Tiempo total estimado: 0 horas, 1 minutos y 0 segundos
Combinación número: 10, Máxima Audiencia encontrada: 2.04
Tiempo total de ejecución: 0 segundos
Tiempo total estimado: 0 horas, 0 minutos y 36 segundos
Combinación número: 15, Máxima Audiencia encontrada: 2.16
Tiempo total de ejecución: 0 segundos
Tiempo total estimado: 0 horas, 0 minutos y 31 segundos
Combinación número: 16, Máxima Audiencia encontrada: 2.

,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,S18,1.30,0.80,1.0400,1.0400
1,Valencia - R.Sociedad,B-A,D16,1.30,0.75,0.9750,0.9750
2,Mallorca - Eibar,C-C,D12,0.47,0.45,0.2115,0.2115
3,Athletic - Barcelona,B-A,S20,1.30,1.00,1.3000,1.3000
4,Leganés - Osasuna,C-C,V20,0.47,0.40,0.1880,0.1880
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,S16,0.90,0.70,0.6300,0.6300


La audiencia total esperada es: 4.757000000000001 M. 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,S20,1.30,1.00,1.3000,1.3000
1,Valencia - R.Sociedad,B-A,D16,1.30,0.75,0.9750,0.9750
2,Mallorca - Eibar,C-C,V20,0.47,0.40,0.1880,0.1880
3,Athletic - Barcelona,B-A,S18,1.30,0.80,1.0400,1.0400
4,Leganés - Osasuna,C-C,D12,0.47,0.45,0.2115,0.2115
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,S16,0.90,0.70,0.6300,0.6300


La audiencia total esperada es: 4.757000000000001 M. 



...

La audiencia total esperada es: 4.757000000000001 M. 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,D16,1.30,0.75,0.9750,0.9750
1,Valencia - R.Sociedad,B-A,S20,1.30,1.00,1.3000,1.3000
2,Mallorca - Eibar,C-C,V20,0.47,0.40,0.1880,0.1880
3,Athletic - Barcelona,B-A,S18,1.30,0.80,1.0400,1.0400
4,Leganés - Osasuna,C-C,D12,0.47,0.45,0.2115,0.2115
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,S16,0.90,0.70,0.6300,0.6300


La audiencia total esperada es: 4.757000000000001 M. 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,D16,1.30,0.75,0.9750,0.9750
1,Valencia - R.Sociedad,B-A,S20,1.30,1.00,1.3000,1.3000
2,Mallorca - Eibar,C-C,D12,0.47,0.45,0.2115,0.2115
3,Athletic - Barcelona,B-A,S18,1.30,0.80,1.0400,1.0400
4,Leganés - Osasuna,C-C,V20,0.47,0.40,0.1880,0.1880
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,S16,0.90,0.70,0.6300,0.6300


La audiencia total esperada es: 4.757000000000001 M. 



**Calcula la complejidad del algoritmo por fuerza bruta**

O(2^n)

**(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta**

## Backtraking con poda

In [ ]:
# Condiciones iniciales para un problema fijo (P = PARTIDOS, H = HORARIOS)

#############################################
  # PROBLEMA P = 8 H = 8
#############################################


lista_partidos = [
      'Celta - Real Madrid',
      'Valencia - R.Sociedad',
      'Mallorca - Eibar',
      'Athletic - Barcelona',
      'Leganés - Osasuna',
      'Villareal - Granada',
      'Alavés - Levante',
      'Espayol - Sevilla',
  ]

lista_categorias = [
      'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B'
  ]

lista_bases = [
      1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9
  ]


Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                  'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85}

In [ ]:
import numpy as np
import pandas as pd
from itertools import product
import time
from collections import Counter

In [ ]:
# Función que devuelve una lista con las diferentes correciones en base a la coincidencia
def Correcion_por_coincidencia(lista_horario_partido):
    #Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]} cogemos directamente la lista para no llamar al diccionario
    correcciones = [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]
    contador = Counter(lista_horario_partido)

    # Aplicar la corrección correspondiente a cada valor
    lista_correciones = [correcciones[min(contador[hora] - 1, len(correcciones) - 1)] for hora in lista_horario_partido]

    return np.array(lista_correciones)

In [ ]:
# Función que genera combinación con restrincciones

def combinaciones_validas(P, H):
  matriz = []
  for comb in product(range(H), repeat=P):
      # if 0 in comb and (H - 1) in comb:
      #   print(comb)
          # yield comb
      if comb[0] == 0 and comb[len(comb)-1] == H-1:
        matriz.append(comb)
  return matriz
matriz_padre = combinaciones_validas(7, 7)

# coger todas las combinaciones que tiene (0,_,2)


comun = (0, 0, 0, 0, 0)
matriz_hijos = (0, 0, 0, 0, 0)

filtered_data = [item for item in matriz_padre if item[:len(matriz_hijos)] == matriz_hijos]
print(filtered_data)

[(0, 0, 0, 0, 0, 0, 6), (0, 0, 0, 0, 0, 1, 6), (0, 0, 0, 0, 0, 2, 6), (0, 0, 0, 0, 0, 3, 6), (0, 0, 0, 0, 0, 4, 6), (0, 0, 0, 0, 0, 5, 6), (0, 0, 0, 0, 0, 6, 6)]


In [ ]:
# Funcion que define si una tupla (0,0,0,0,0,0,0) comienza por (0,0,0,0)
tupla = (0, 0, 0, 0, 0, 0, 6)
comienzo = (0, 0, 0, 0)
comienzo in tupla

False

In [ ]:
# Función que genera dataframe
def generar_dataframe(lista_partidos: list, lista_categorias: list, lista_horario_partido:list, lista_bases: list, lista_ponderaciones:list, lista_base_ponderacion:list, lista_correciones_corregida:list ):

  dict_jornada = {
      'Partido' : lista_partidos,
      'Categorías' : lista_categorias,
      'Horario' : lista_horario_partido,
      'Base(Mill.)' : lista_bases,
      'Ponderación' : lista_ponderaciones,
      'Base*Ponderación' : lista_base_ponderacion,
      'Correción Coincidencia' : lista_correciones_corregida,
  }

  df_jornada = pd.DataFrame(dict_jornada)
  return df_jornada

In [ ]:
Total_max = 0
horario_max_audiencia = []

listado_horarios = list(Ponderacion.keys())

P = len(lista_partidos)
H = len(listado_horarios)
combinaciones_totales = (H**P - (H-1)**P - (H-1)**P + (H-2)**P)

# Generador de combinaciones teniendo en cuentra restrincciones
gen = combinaciones_validas(P, H)

start_time = time.time()  # Inicia temporizador
# Cálculo de la audiencia para cada combinación posible
for i, fila in enumerate(gen):
  fila_array = np.array(fila)
  lista_horario_partido = np.array(listado_horarios)[fila_array] # lista de listas con todos los horarios posibles para la jornada
  lista_ponderaciones = np.array([Ponderacion[h] for h in lista_horario_partido])
  lista_base_ponderacion = np.array(lista_bases)*lista_ponderaciones
  lista_correciones = Correcion_por_coincidencia(lista_horario_partido)
  lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)
  Total = np.sum(lista_correciones_corregida)

  if Total > Total_max:
    Total_max = Total
    #horario_max_audiencia = [] # vaciamos la lista
    horario_max_audiencia = [lista_horario_partido.tolist()]
    print(f'Combinación número: {i}, Máxima Audiencia encontrada: {Total_max:.2f}')
    end_time = time.time()  # Finaliza temporizador
    print(f"Tiempo total de ejecución: {int(end_time - start_time)} segundos")

    if i>1:
      tiempo_estimado_total = (combinaciones_totales*(end_time - start_time)/i)
      horas = int(tiempo_estimado_total//3600)
      minutos = int((tiempo_estimado_total %3600)//60)
      segundos = int(tiempo_estimado_total-horas*3600)%60
      print(f"Tiempo total estimado: {horas} horas, {minutos} minutos y {segundos} segundos")

  elif Total == Total_max:
    horario_max_audiencia.append(lista_horario_partido.tolist())

# ¿1 Solución o varias?
if len(horario_max_audiencia) == 1:
  print(f'Solución: El horario con máxima audiencia es: {horario_max_audiencia} y obtiene una audiencia de {Total_max} \n')
elif len(horario_max_audiencia) > 1:
  print(f'Solución: Los horarios con máxima audiencia son: {len(horario_max_audiencia)} y se obtiene una audiencia de {Total_max} en cualquiera de estas combinaciones \n')

In [ ]:
# Mostramos el DataFrame para los horarios que producen una máxima audiencia en una jornada
for lista_horario_partido in horario_max_audiencia:
  lista_ponderaciones = np.array([Ponderacion[h] for h in lista_horario_partido])
  lista_base_ponderacion = np.array(lista_bases)*lista_ponderaciones
  lista_correciones = Correcion_por_coincidencia(lista_horario_partido)
  lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)
  df_jornada_max_audiencia = generar_dataframe(lista_partidos, lista_categorias, lista_horario_partido, lista_bases, lista_ponderaciones, lista_base_ponderacion, lista_correciones_corregida)
  display(df_jornada_max_audiencia)
  Total = np.sum(lista_correciones_corregida)
  print(f'La audiencia total esperada es: {Total} M. \n')

## Optimización estructuras de datos del problema

In [ ]:
def problema():
  import numpy as np
  import pandas as pd
  from itertools import product
  import time


  # Función que devuelve una lista con las diferentes correciones en base a la coincidencia
  def Correcion_por_coincidencia(lista_horario_partido):
    Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
    lista_repeticiones = []

    unique, counts = np.unique(lista_horario_partido, return_counts=True)
    dict_valores = dict(zip(unique, counts))

    for valor in lista_horario_partido:
      repeticiones = dict_valores[valor] - 1 # restamos 1 porque si está le corresponde una correción de coincidencia de -0% no del -25%
      lista_repeticiones.append(repeticiones)

    lista_correciones = np.array(Coincidencias['-%'])[np.array(lista_repeticiones)]
    return lista_correciones

  # Función que genera dataframe
  def generar_dataframe(lista_partidos: list, lista_categorias: list, lista_horario_partido:list, lista_bases: list, lista_ponderaciones:list, lista_base_ponderacion:list, lista_correciones_corregida:list ):

    dict_jornada = {
        'Partido' : lista_partidos,
        'Categorías' : lista_categorias,
        'Horario' : lista_horario_partido,
        'Base(Mill.)' : lista_bases,
        'Ponderación' : lista_ponderaciones,
        'Base*Ponderación' : lista_base_ponderacion,
        'Correción Coincidencia' : lista_correciones_corregida,
    }

    df_jornada = pd.DataFrame(dict_jornada)
    return df_jornada

  #Función que genera combinación con restrincciones
  def combinaciones_validas(P, H):
      for comb in product(range(H), repeat=P):
          if 0 in comb and (H - 1) in comb:
              yield comb

  # Condiciones iniciales para un problema fijo (P = PARTIDOS, H = HORARIOS)

  #############################################
  # PROBLEMA P = 4 H = 3
  #############################################

  # lista_partidos = [
  #     'Celta - Real Madrid',
  #     'Valencia - R.Sociedad',
  #     'Mallorca - Eibar',
  #     'Athletic - Barcelona',
  # ]

  # lista_categorias = ['B-A', 'B-A', 'C-C', 'B-A']

  # lista_bases = [1.3, 1.3, 0.47, 1.3]

  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7}

  #############################################
  # PROBLEMA P = 7 H = 7
  #############################################
  # lista_partidos = [
  #       'Celta - Real Madrid',
  #       'Valencia - R.Sociedad',
  #       'Mallorca - Eibar',
  #       'Athletic - Barcelona',
  #       'Leganés - Osasuna',
  #       'Villareal - Granada',
  #       'Alavés - Levante',
  #   ]

  # lista_categorias = [
  #       'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B'
  #   ]

  # lista_bases = [
  #       1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9
  #   ]


  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
  #                  'S20' : 1, 'D12' : 0.45, 'D16' : 0.75}

  #############################################
    # PROBLEMA P = 8 H = 8
  #############################################


  lista_partidos = [
        'Celta - Real Madrid',
        'Valencia - R.Sociedad',
        'Mallorca - Eibar',
        'Athletic - Barcelona',
        'Leganés - Osasuna',
        'Villareal - Granada',
        'Alavés - Levante',
        'Espayol - Sevilla',
    ]

  lista_categorias = [
        'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B'
    ]

  lista_bases = [
        1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9
    ]


  Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                   'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85}

  #############################################
    # PROBLEMA P = 10 H = 10
  #############################################

  # lista_partidos = [
  #   'Celta - Real Madrid',
  #   'Valencia - R.Sociedad',
  #   'Mallorca - Eibar',
  #   'Athletic - Barcelona',
  #   'Leganés - Osasuna',
  #   'Villareal - Granada',
  #   'Alavés - Levante',
  #   'Espayol - Sevilla',
  #   'Betis - Valladolid',
  #   'Atlético - Getafe',
  #   ]

  # lista_categorias = [
  #     'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B', 'B-C', 'B-B'
  # ]

  # lista_bases = [
  #     1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75, 0.9
  # ]


  # Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
  #                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
  #                 'D20' : 1, 'L20' : 0.4}

  ##############################################

  Total_max = 0
  horario_max_audiencia = []

  listado_horarios = list(Ponderacion.keys())

  P = len(lista_partidos)
  H = len(listado_horarios)

  # Generador de combinaciones teniendo en cuentra restrincciones
  gen = combinaciones_validas(P, H)

  start_time = time.time()  # Inicia temporizador
  # Cálculo de la audiencia para cada combinación posible
  for i, fila in enumerate(gen):
    lista_horario_partido = np.array(listado_horarios)[np.array(fila)] # lista de listas con todos los horarios posibles para la jornada
    lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]
    lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
    lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
    lista_correciones_corregida = lista_base_ponderacion*(np.ones(len(lista_correciones))-lista_correciones)
    Total = np.sum(lista_correciones_corregida)

    if Total > Total_max:
      Total_max = Total
      horario_max_audiencia = [] # vaciamos la lista
      horario_max_audiencia = [lista_horario_partido.tolist()]
      print(f'Combinación número: {i}, Máxima Audiencia encontrada: {Total_max}')
      end_time = time.time()  # Finaliza temporizador
      print(f"Tiempo total de ejecución: {end_time - start_time:} segundos")
      if i>1:
        tiempo_total = ((H**P - (H-1)**P - (H-1)**P + (H-2)**P)*(end_time - start_time)/i)
        horas = tiempo_total//3600
        minutos = (tiempo_total-horas*3600)//60
        segundos = int(tiempo_total-horas*3600)%60
        print(f"Tiempo total estimado: {horas} horas, {minutos} minutos y {segundos} segundos")
    elif Total == Total_max:
      horario_max_audiencia.append(lista_horario_partido.tolist())

  # ¿1 Solución o varias?
  if len(horario_max_audiencia) == 1:
    print(f'Solución: El horario con máxima audiencia es: {horario_max_audiencia} y obtiene una audiencia de {Total_max} \n')
  elif len(horario_max_audiencia) > 1:
    print(f'Solución: Los horarios con máxima audiencia son: {len(horario_max_audiencia)} y se obtiene una audiencia de {Total_max} en cualquiera de estas combinaciones \n')

  # Mostramos el DataFrame para los horarios que producen una máxima audiencia en una jornada
  for lista_horario_partido in horario_max_audiencia:
    lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]
    lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
    lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
    lista_correciones_corregida = lista_base_ponderacion*(np.ones(len(lista_correciones))-lista_correciones)
    df_jornada_max_audiencia = generar_dataframe(lista_partidos, lista_categorias, lista_horario_partido, lista_bases, lista_ponderaciones, lista_base_ponderacion, lista_correciones_corregida)
    print(df_jornada_max_audiencia)
    Total = np.sum(lista_correciones_corregida)
    print(f'La audiencia total esperada es: {Total} M. \n')

In [ ]:
problema()

Combinación número: 0, Máxima Audiencia encontrada: 1.33612
Tiempo total de ejecución: 0.00043582916259765625 segundos
Combinación número: 1, Máxima Audiencia encontrada: 1.8190000000000002
Tiempo total de ejecución: 0.0006046295166015625 segundos
Combinación número: 2, Máxima Audiencia encontrada: 1.9540000000000002
Tiempo total de ejecución: 0.0007326602935791016 segundos
Tiempo total estimado: 0.0 horas, 42.0 minutos y 17 segundos
Combinación número: 3, Máxima Audiencia encontrada: 2.044
Tiempo total de ejecución: 0.0008800029754638672 segundos
Tiempo total estimado: 0.0 horas, 33.0 minutos y 51 segundos
Combinación número: 4, Máxima Audiencia encontrada: 2.224
Tiempo total de ejecución: 0.0010998249053955078 segundos
Tiempo total estimado: 0.0 horas, 31.0 minutos y 44 segundos
Combinación número: 17, Máxima Audiencia encontrada: 2.3883
Tiempo total de ejecución: 0.002453327178955078 segundos
Tiempo total estimado: 0.0 horas, 16.0 minutos y 39 segundos
Combinación número: 18, Máxima

KeyboardInterrupt: 

In [ ]:
0.0006723403930664062 * 8**8/3600

0.0032472610473632812/28

0.0001159736088344029

## PROBLEMA

In [ ]:
import numpy as np
import pandas as pd
from itertools import product

# Función que devuelve una lista con las diferentes correciones en base a la coincidencia
def Correcion_por_coincidencia(lista_horario_partido):
  Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
  lista_repeticiones = []

  unique, counts = np.unique(lista_horario_partido, return_counts=True)
  dict_valores = dict(zip(unique, counts))

  for valor in lista_horario_partido:
    repeticiones = dict_valores[valor] - 1 # restamos 1 porque si está le corresponde una correción de coincidencia de -0% no del -25%
    lista_repeticiones.append(repeticiones)

  lista_correciones = np.array(Coincidencias['-%'])[np.array(lista_repeticiones)]
  return lista_correciones

# Función que genera dataframe
def generar_dataframe(lista_partidos: list, lista_categorias: list, lista_horario_partido:list, lista_bases: list, lista_ponderaciones:list, lista_base_ponderacion:list, lista_correciones_corregida:list ):

  dict_jornada = {
      'Partido' : lista_partidos,
      'Categorías' : lista_categorias,
      'Horario' : lista_horario_partido,
      'Base(Mill.)' : lista_bases,
      'Ponderación' : lista_ponderaciones,
      'Base*Ponderación' : lista_base_ponderacion,
      'Correción Coincidencia' : lista_correciones_corregida,
  }

  df_jornada = pd.DataFrame(dict_jornada)
  return df_jornada

#Función que genera combinación con restrincciones
def combinaciones_validas(P, H):
    for comb in product(range(H), repeat=P):
        if 0 in comb and (H - 1) in comb:
            yield comb

# Condiciones iniciales para un problema fijo
#############################################
lista_partidos = [
      'Celta - Real Madrid',
      'Valencia - R.Sociedad',
      'Mallorca - Eibar',
      'Athletic - Barcelona',
      'Leganés - Osasuna',
      'Villareal - Granada',
      'Alavés - Levante',
      'Espayol - Sevilla',
  ]

lista_categorias = [
      'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B'
  ]

lista_bases = [
      1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9
  ]


Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85}

lista_totales = []
dict_resultados = {}
Total_max = 0
horario_max_audiencia = []

##############################################

listado_horarios = list(Ponderacion.keys())

P = len(lista_partidos)
H = len(listado_horarios)

# Generador de combinaciones teniendo en cuentra restrincciones
gen = combinaciones_validas(P, H)

# Cálculo de la audiencia para cada combinación posible
for i, fila in enumerate(gen):
  lista_horario_partido = np.array(listado_horarios)[np.array(fila)] # lista de listas con todos los horarios posibles para la jornada
  lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]
  lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
  lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
  lista_correciones_corregida = lista_base_ponderacion*(np.ones(len(lista_correciones))-lista_correciones)
  Total = np.sum(lista_correciones_corregida)
  if Total > Total_max:
    Total_max = Total
    horario_max_audiencia = [] # vaciamos la lista
    horario_max_audiencia = [lista_horario_partido.tolist()]
    print(i)
  elif Total == Total_max:
    horario_max_audiencia.append(lista_horario_partido.tolist())

# ¿1 Solución o varias?
if len(horario_max_audiencia) == 1:
  print(f'Solución: El horario con máxima audiencia es: {horario_max_audiencia} y obtiene una audiencia de {Total_max} \n')
elif len(horario_max_audiencia) > 1:
  print(f'Solución: Los horarios con máxima audiencia son: {horario_max_audiencia} y se obtiene una audiencia de {Total_max} en cualquiera de estas combinaciones \n')

# Mostramos el DataFrame para los horarios que producen una máxima audencia en una jornada
for lista_horario_partido in horario_max_audiencia:
  lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]
  lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
  lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
  lista_correciones_corregida = lista_base_ponderacion*(np.ones(len(lista_correciones))-lista_correciones)
  df_jornada_max_audiencia = generar_dataframe(lista_partidos, lista_categorias, lista_horario_partido, lista_bases, lista_ponderaciones, lista_base_ponderacion, lista_correciones_corregida)
  display(df_jornada_max_audiencia)
  Total = np.sum(lista_correciones_corregida)
  print(f'La audiencia total esperada es: {Total} M. \n')

0
1
2
3
4
17
18
19
34
49
188
202
203
218
387
601
1063
1729
1744
1898
1913
2067
2082
2296
2758
3608
3963
3991
4453
5489
5517
7000
7042
7169
7211
19924
19952
20414
21450
21478
23130
23172
148458
148486
148948
149984
150012
151664
151706
167667
181906
181934
182413
182441
183586
183628
184093
184135
310947
310975
312627
312669
457107
608524
1499282
1499324
1572769
1653193
2302140
2302756
2383131
3111333
3113125
Solución: Los horarios con máxima audiencia son: [['S18', 'S20', 'V20', 'D18', 'D12', 'S12', 'S16', 'D16'], ['S18', 'S20', 'V20', 'D18', 'D12', 'S12', 'D16', 'S16'], ['S18', 'S20', 'D12', 'D18', 'V20', 'S12', 'S16', 'D16'], ['S18', 'S20', 'D12', 'D18', 'V20', 'S12', 'D16', 'S16'], ['S18', 'D18', 'V20', 'S20', 'D12', 'S12', 'S16', 'D16'], ['S18', 'D18', 'V20', 'S20', 'D12', 'S12', 'D16', 'S16'], ['S18', 'D18', 'D12', 'S20', 'V20', 'S12', 'S16', 'D16'], ['S18', 'D18', 'D12', 'S20', 'V20', 'S12', 'D16', 'S16'], ['S20', 'S18', 'V20', 'D18', 'D12', 'S12', 'S16', 'D16'], ['S20', 'S18', '

,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,S18,1.30,0.80,1.0400,1.0400
1,Valencia - R.Sociedad,B-A,S20,1.30,1.00,1.3000,1.3000
2,Mallorca - Eibar,C-C,V20,0.47,0.40,0.1880,0.1880
3,Athletic - Barcelona,B-A,D18,1.30,0.85,1.1050,1.1050
4,Leganés - Osasuna,C-C,D12,0.47,0.45,0.2115,0.2115
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,S16,0.90,0.70,0.6300,0.6300
7,Espayol - Sevilla,B-B,D16,0.90,0.75,0.6750,0.6750


La audiencia total esperada es: 5.562 M. 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,S18,1.30,0.80,1.0400,1.0400
1,Valencia - R.Sociedad,B-A,S20,1.30,1.00,1.3000,1.3000
2,Mallorca - Eibar,C-C,V20,0.47,0.40,0.1880,0.1880
3,Athletic - Barcelona,B-A,D18,1.30,0.85,1.1050,1.1050
4,Leganés - Osasuna,C-C,D12,0.47,0.45,0.2115,0.2115
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,D16,0.90,0.75,0.6750,0.6750
7,Espayol - Sevilla,B-B,S16,0.90,0.70,0.6300,0.6300


La audiencia total esperada es: 5.562 M. 



...

La audiencia total esperada es: 5.562 M. 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,D18,1.30,0.85,1.1050,1.1050
1,Valencia - R.Sociedad,B-A,S20,1.30,1.00,1.3000,1.3000
2,Mallorca - Eibar,C-C,D12,0.47,0.45,0.2115,0.2115
3,Athletic - Barcelona,B-A,S18,1.30,0.80,1.0400,1.0400
4,Leganés - Osasuna,C-C,V20,0.47,0.40,0.1880,0.1880
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,S16,0.90,0.70,0.6300,0.6300
7,Espayol - Sevilla,B-B,D16,0.90,0.75,0.6750,0.6750


La audiencia total esperada es: 5.562 M. 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,D18,1.30,0.85,1.1050,1.1050
1,Valencia - R.Sociedad,B-A,S20,1.30,1.00,1.3000,1.3000
2,Mallorca - Eibar,C-C,D12,0.47,0.45,0.2115,0.2115
3,Athletic - Barcelona,B-A,S18,1.30,0.80,1.0400,1.0400
4,Leganés - Osasuna,C-C,V20,0.47,0.40,0.1880,0.1880
5,Villareal - Granada,B-C,S12,0.75,0.55,0.4125,0.4125
6,Alavés - Levante,B-B,D16,0.90,0.75,0.6750,0.6750
7,Espayol - Sevilla,B-B,S16,0.90,0.70,0.6300,0.6300


La audiencia total esperada es: 5.562 M. 



## Cálculo de audiencia de problema fijo

In [ ]:
import numpy as np
import pandas as pd

def Correcion_por_coincidencia(lista_horario_partido): # devuelve una lista con las diferentes correciones en base a la coincidencia
  Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
  lista_correciones = []

  for hora in lista_horario_partido:
    contador = lista_horario_partido.count(hora) - 1
    lista_correciones.append(Coincidencias['-%'][contador])
  return lista_correciones

# Condiciones iniciales para un problema fijo
lista_partidos = [
    'Celta - Real Madrid',
    'Valencia - R.Sociedad',
    'Mallorca - Eibar',
    'Athletic - Barcelona',
    'Leganés - Osasuna',
    'Villareal - Granada',
    'Alavés - Levante',
    'Espayol - Sevilla',
    'Betis - Valladolid',
    'Atlético - Getafe',
]

lista_categorias = ['B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B', 'B-C', 'B-B']

lista_bases = [1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75, 0.9]

Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                'D20' : 1, 'L20' : 0.4}

listado_horarios = list(Ponderacion.keys())

lista_horario_partido = ['V20', 'S12', 'S16', 'S18', 'S20', 'D16', 'D16', 'D18', 'D20', 'L20']
lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]

lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)

dict_jornada = {
    'Partido' : lista_partidos,
    'Categorías' : lista_categorias,
    'Horario' : lista_horario_partido,
    'Base(Mill.)' : lista_bases,
    'Ponderación' : lista_ponderaciones,
    'Base*Ponderación' : lista_base_ponderacion,
    'Correción Coincidencia' : lista_correciones_corregida,
}

df_jornada = pd.DataFrame(dict_jornada)
display(df_jornada)
Total = np.sum(lista_correciones_corregida)
print('\n Total: ',Total)

,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,V20,1.30,0.40,0.5200,0.520000
1,Valencia - R.Sociedad,B-A,S12,1.30,0.55,0.7150,0.715000
2,Mallorca - Eibar,C-C,S16,0.47,0.70,0.3290,0.329000
3,Athletic - Barcelona,B-A,S18,1.30,0.80,1.0400,1.040000
4,Leganés - Osasuna,C-C,S20,0.47,1.00,0.4700,0.470000
5,Villareal - Granada,B-C,D16,0.75,0.75,0.5625,0.421875
6,Alavés - Levante,B-B,D16,0.90,0.75,0.6750,0.506250
7,Espayol - Sevilla,B-B,D18,0.90,0.85,0.7650,0.765000
8,Betis - Valladolid,B-C,D20,0.75,1.00,0.7500,0.750000
9,Atlético - Getafe,B-B,L20,0.90,0.40,0.3600,0.360000



 Total:  5.877125


In [ ]:
# Creación de función que convierte matriz de combinaciones en los distintos horarios
from itertools import product

lista_partidos = [
    'Celta - Real Madrid',
    'Valencia - R.Sociedad',
    'Mallorca - Eibar',
    'Athletic - Barcelona',
    'Leganés - Osasuna',
    'Villareal - Granada',
    'Alavés - Levante',
    'Espayol - Sevilla',
    'Betis - Valladolid',
    'Atlético - Getafe',
]

Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                'D20' : 1, 'L20' : 0.4}

listado_horarios = list(Ponderacion.keys())

P = len(lista_partidos)
H = len(listado_horarios)

# Generar todas las combinaciones de P números del 0 a H-1
matriz = list(product(range(H), repeat=P))
matriz_restringida = [tuple for tuple in matriz if (0 in tuple and H-1 in tuple)]
matriz_restringida


## SUB-PROBLEMA OPTIMIZADO CON GENERADOR DE COMBINACIONES Y SIN GUARDADO DE ESTRUCTURAS INNECESARIAS

In [ ]:
import numpy as np
import pandas as pd
from itertools import product

# Función que devuelve una lista con las diferentes correciones en base a la coincidencia
def Correcion_por_coincidencia(lista_horario_partido):
  Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
  lista_repeticiones = []

  unique, counts = np.unique(lista_horario_partido, return_counts=True)
  dict_valores = dict(zip(unique, counts))

  for valor in lista_horario_partido:
    repeticiones = dict_valores[valor] - 1 # restamos 1 porque si está le corresponde una correción de coincidencia de -0% no del -25%
    lista_repeticiones.append(repeticiones)

  lista_correciones = np.array(Coincidencias['-%'])[np.array(lista_repeticiones)]
  return lista_correciones

# Función que genera dataframe
def generar_dataframe(lista_partidos: list, lista_categorias: list, lista_horario_partido:list, lista_bases: list, lista_ponderaciones:list, lista_base_ponderacion:list, lista_correciones_corregida:list ):

  dict_jornada = {
      'Partido' : lista_partidos,
      'Categorías' : lista_categorias,
      'Horario' : lista_horario_partido,
      'Base(Mill.)' : lista_bases,
      'Ponderación' : lista_ponderaciones,
      'Base*Ponderación' : lista_base_ponderacion,
      'Correción Coincidencia' : lista_correciones_corregida,
  }

  df_jornada = pd.DataFrame(dict_jornada)
  return df_jornada

#Función que genera combinación con restrincciones
def combinaciones_validas(P, H):
    for comb in product(range(H), repeat=P):
        if 0 in comb and (H - 1) in comb:
            yield comb

# Condiciones iniciales para un problema fijo
##############################################
# lista_partidos = [
#       'Celta - Real Madrid',
#       'Valencia - R.Sociedad',
#       'Mallorca - Eibar',
#       'Athletic - Barcelona',
#       'Leganés - Osasuna',
#       'Villareal - Granada',
#       'Alavés - Levante',
#       'Espayol - Sevilla',
#   ]

# lista_categorias = [
#       'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B'
#   ]

# lista_bases = [
#       1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9
#   ]


# Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
#                  'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85}

lista_partidos = [
    'Celta - Real Madrid',
    'Valencia - R.Sociedad',
    'Mallorca - Eibar',
    'Athletic - Barcelona',
]

lista_categorias = ['B-A', 'B-A', 'C-C', 'B-A']

lista_bases = [1.3, 1.3, 0.47, 1.3]

Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7}

lista_totales = []
dict_resultados = {}
Total_max = 0
horario_max_audiencia = []

##############################################

listado_horarios = list(Ponderacion.keys())

P = len(lista_partidos)
H = len(listado_horarios)

# Generador de combinaciones teniendo en cuentra restrincciones
gen = combinaciones_validas(P, H)

# Cálculo de la audiencia para cada combinación posible
for i, fila in enumerate(gen):
  lista_horario_partido = np.array(listado_horarios)[np.array(fila)] # lista de listas con todos los horarios posibles para la jornada
  lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]
  lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
  lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
  lista_correciones_corregida = lista_base_ponderacion*(np.ones(len(lista_correciones))-lista_correciones)
  Total = np.sum(lista_correciones_corregida)
  if Total > Total_max:
    Total_max = Total
    horario_max_audiencia = [] # vaciamos la lista
    horario_max_audiencia = [lista_horario_partido.tolist()]
  elif Total == Total_max:
    horario_max_audiencia.append(lista_horario_partido.tolist())

# ¿1 Solución o varias?
if len(horario_max_audiencia) == 1:
  print(f'Solución: El horario con máxima audiencia es: {horario_max_audiencia} y obtiene una audiencia de {Total_max} \n')
elif len(horario_max_audiencia) > 1:
  print(f'Solución: Los horarios con máxima audiencia son: {horario_max_audiencia} y se obtiene una audiencia de {Total_max} en cualquiera de estas combinaciones \n')

# Mostramos el DataFrame para los horarios que producen una máxima audencia en una jornada
for lista_horario_partido in horario_max_audiencia:
  lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]
  lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
  lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
  lista_correciones_corregida = lista_base_ponderacion*(np.ones(len(lista_correciones))-lista_correciones)
  df_jornada_max_audiencia = generar_dataframe(lista_partidos, lista_categorias, lista_horario_partido, lista_bases, lista_ponderaciones, lista_base_ponderacion, lista_correciones_corregida)
  display(df_jornada_max_audiencia)
  Total = np.sum(lista_correciones_corregida)
  print(f'La audiencia total esperada es: {Total} M. \n')

Solución: Los horarios con máxima audiencia son: [['S12', 'S16', 'V20', 'S16'], ['S16', 'S12', 'V20', 'S16'], ['S16', 'S16', 'V20', 'S12']] y se obtiene una audiencia de 2.268 en cualquiera de estas combinaciones 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,S12,1.30,0.55,0.715,0.7150
1,Valencia - R.Sociedad,B-A,S16,1.30,0.70,0.910,0.6825
2,Mallorca - Eibar,C-C,V20,0.47,0.40,0.188,0.1880
3,Athletic - Barcelona,B-A,S16,1.30,0.70,0.910,0.6825


La audiencia total esperada es: 2.268 M. 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,S16,1.30,0.70,0.910,0.6825
1,Valencia - R.Sociedad,B-A,S12,1.30,0.55,0.715,0.7150
2,Mallorca - Eibar,C-C,V20,0.47,0.40,0.188,0.1880
3,Athletic - Barcelona,B-A,S16,1.30,0.70,0.910,0.6825


La audiencia total esperada es: 2.268 M. 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,S16,1.30,0.70,0.910,0.6825
1,Valencia - R.Sociedad,B-A,S16,1.30,0.70,0.910,0.6825
2,Mallorca - Eibar,C-C,V20,0.47,0.40,0.188,0.1880
3,Athletic - Barcelona,B-A,S12,1.30,0.55,0.715,0.7150


La audiencia total esperada es: 2.268 M. 



## SUB-PROBLEMA

### Mostrando todos los dataframes

In [ ]:
import numpy as np
import pandas as pd
from itertools import product

# Función que devuelve una lista con las diferentes correciones en base a la coincidencia
def Correcion_por_coincidencia(lista_horario_partido):
  Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
  lista_repeticiones = []

  unique, counts = np.unique(lista_horario_partido, return_counts=True)
  dict_valores = dict(zip(unique, counts))

  for valor in lista_horario_partido:
    repeticiones = dict_valores[valor] - 1 # restamos 1 porque si está le corresponde una correción de coincidencia de -0% no del -25%
    lista_repeticiones.append(repeticiones)

  lista_correciones = np.array(Coincidencias['-%'])[np.array(lista_repeticiones)]
  return lista_correciones

# Condiciones iniciales para un problema fijo
##############################################
lista_partidos = [
    'Celta - Real Madrid',
    'Valencia - R.Sociedad',
    'Mallorca - Eibar',
    'Athletic - Barcelona',
]

lista_categorias = ['B-A', 'B-A', 'C-C', 'B-A']

lista_bases = [1.3, 1.3, 0.47, 1.3]

Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7}

lista_totales = []
dict_resultados = {}

##############################################

listado_horarios = list(Ponderacion.keys())

P = len(lista_partidos)
H = len(listado_horarios)

# Generar todas las combinaciones de P números del 0 a H-1
matriz = list(product(range(H), repeat=P))
matriz_restringida = [comb for comb in matriz if (0 in comb and H-1 in comb)]

# Convertir la matriz de combinaciones con restrincciones a horarios
horarios = np.array(listado_horarios)[np.array(matriz_restringida)] # lista de listas con todos los horarios posibles para la jornada

for lista_horario_partido in horarios:
  lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]
  lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
  lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
  lista_correciones_corregida = lista_base_ponderacion*(np.ones(len(lista_correciones))-lista_correciones)

  df, Total = generar_dataframe(lista_partidos, lista_categorias, lista_horario_partido, lista_bases, lista_ponderaciones, lista_base_ponderacion, lista_correciones_corregida)
  display(df)
  print(Total,'\n')
  lista_totales.append(Total)

lista_totales = np.array(lista_totales)
dict_resultados = dict(zip(lista_totales, [tuple(h) for h in horarios]))
horario_max_audiencia = dict_resultados.get(np.max(lista_totales))
horario_max_audiencia = [str(x) for x in horario_max_audiencia]
print(f'Solución: El horario con máxima audiencia es: {horario_max_audiencia} y obtiene una audiencia de {np.max(lista_totales)}')





,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,V20,1.30,0.4,0.520,0.2860
1,Valencia - R.Sociedad,B-A,V20,1.30,0.4,0.520,0.2860
2,Mallorca - Eibar,C-C,V20,0.47,0.4,0.188,0.1034
3,Athletic - Barcelona,B-A,S16,1.30,0.7,0.910,0.9100


1.5854 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,V20,1.30,0.40,0.5200,0.3900
1,Valencia - R.Sociedad,B-A,V20,1.30,0.40,0.5200,0.3900
2,Mallorca - Eibar,C-C,S12,0.47,0.55,0.2585,0.2585
3,Athletic - Barcelona,B-A,S16,1.30,0.70,0.9100,0.9100


1.9485 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,V20,1.30,0.4,0.520,0.286
1,Valencia - R.Sociedad,B-A,V20,1.30,0.4,0.520,0.286
2,Mallorca - Eibar,C-C,S16,0.47,0.7,0.329,0.329
3,Athletic - Barcelona,B-A,V20,1.30,0.4,0.520,0.286


...

1.6894999999999998 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,S16,1.30,0.70,0.9100,0.6825
1,Valencia - R.Sociedad,B-A,S16,1.30,0.70,0.9100,0.6825
2,Mallorca - Eibar,C-C,S12,0.47,0.55,0.2585,0.2585
3,Athletic - Barcelona,B-A,V20,1.30,0.40,0.5200,0.5200


2.1434999999999995 



,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,S16,1.30,0.7,0.910,0.50050
1,Valencia - R.Sociedad,B-A,S16,1.30,0.7,0.910,0.50050
2,Mallorca - Eibar,C-C,S16,0.47,0.7,0.329,0.18095
3,Athletic - Barcelona,B-A,V20,1.30,0.4,0.520,0.52000


1.7019499999999999 

Solución: El horario con máxima audiencia es: ['S16', 'S16', 'V20', 'S12'] y obtiene una audiencia de 2.268


In [ ]:
lista_totales = np.array(lista_totales)
dict_resultados = dict(zip(lista_totales, [tuple(h) for h in horarios]))
print(dict_resultados, '\n')
print(np.max(lista_totales))
horario_max_audiencia = dict_resultados.get(np.max(lista_totales))
horario_max_audiencia = [str(x) for x in horario_max_audiencia]
horario_max_audiencia

{np.float64(1.5854): (np.str_('S16'), np.str_('V20'), np.str_('V20'), np.str_('V20')), np.float64(1.9485): (np.str_('V20'), np.str_('V20'), np.str_('S12'), np.str_('S16')), np.float64(1.187): (np.str_('V20'), np.str_('V20'), np.str_('S16'), np.str_('V20')), np.float64(1.824): (np.str_('V20'), np.str_('V20'), np.str_('S16'), np.str_('S12')), np.float64(1.70925): (np.str_('S16'), np.str_('V20'), np.str_('S16'), np.str_('V20')), np.float64(2.1559999999999997): (np.str_('S16'), np.str_('V20'), np.str_('V20'), np.str_('S12')), np.float64(2.160125): (np.str_('S16'), np.str_('S12'), np.str_('S12'), np.str_('V20')), np.float64(1.8239999999999998): (np.str_('S12'), np.str_('V20'), np.str_('S16'), np.str_('V20')), np.float64(1.9215000000000002): (np.str_('S12'), np.str_('S12'), np.str_('S16'), np.str_('V20')), np.float64(2.16425): (np.str_('S16'), np.str_('S12'), np.str_('S16'), np.str_('V20')), np.float64(1.8959999999999997): (np.str_('S16'), np.str_('V20'), np.str_('V20'), np.str_('S16')), np.

['S16', 'S16', 'V20', 'S12']

In [ ]:
import numpy as np
import pandas as pd
from itertools import product

# Función que devuelve una lista con las diferentes correciones en base a la coincidencia
def Correcion_por_coincidencia(lista_horario_partido):
  Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
  lista_repeticiones = []

  unique, counts = np.unique(lista_horario_partido, return_counts=True)
  dict_valores = dict(zip(unique, counts))

  for valor in lista_horario_partido:
    repeticiones = dict_valores[valor] - 1 # restamos 1 porque si está le corresponde una correción de coincidencia de -0% no del -25%
    lista_repeticiones.append(repeticiones)

  lista_correciones = np.array(Coincidencias['-%'])[np.array(lista_repeticiones)]
  return lista_correciones

# Condiciones iniciales para un problema fijo
##############################################
lista_partidos = [
    'Celta - Real Madrid',
    'Valencia - R.Sociedad',
    'Mallorca - Eibar',
    'Athletic - Barcelona',
]

lista_categorias = ['B-A', 'B-A', 'C-C', 'B-A']

lista_bases = [1.3, 1.3, 0.47, 1.3]

Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7}

##############################################

listado_horarios = list(Ponderacion.keys())

P = len(lista_partidos)
H = len(listado_horarios)

# Generar todas las combinaciones de P números del 0 a H-1
matriz = list(product(range(H), repeat=P))
matriz_restringida = [comb for comb in matriz if (0 in comb and H-1 in comb)]

# Convertir la matriz de combinaciones con restrincciones a horarios
horarios = np.array(listado_horarios)[np.array(matriz_restringida)] # lista de listas con todos los horarios posibles para la jornada

for lista_horario_partido in horarios:
  print(lista_horario_partido)
  lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]
  print(lista_ponderaciones)
  lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
  print(lista_base_ponderacion)
  lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
  print(lista_correciones)
  lista_correciones_corregida = lista_base_ponderacion*(np.ones(len(lista_correciones))-lista_correciones)
  print(lista_correciones_corregida, '\n')

['V20' 'V20' 'V20' 'S16']
[0.4, 0.4, 0.4, 0.7]
[0.52  0.52  0.188 0.91 ]
[0.45 0.45 0.45 0.  ]
[0.286  0.286  0.1034 0.91  ] 

['V20' 'V20' 'S12' 'S16']
[0.4, 0.4, 0.55, 0.7]
[0.52   0.52   0.2585 0.91  ]
[0.25 0.25 0.   0.  ]
[0.39   0.39   0.2585 0.91  ] 

['V20' 'V20' 'S16' 'V20']
[0.4, 0.4, 0.7, 0.4]
[0.52  0.52  0.329 0.52 ]
[0.45 0.45 0.   0.45]
[0.286 0.286 0.329 0.286] 

['V20' 'V20' 'S16' 'S12']
[0.4, 0.4, 0.7, 0.55]
[0.52  0.52  0.329 0.715]
[0.25 0.25 0.   0.  ]
[0.39  0.39  0.329 0.715] 

['V20' 'V20' 'S16' 'S16']
[0.4, 0.4, 0.7, 0.7]
[0.52  0.52  0.329 0.91 ]
[0.25 0.25 0.25 0.25]
[0.39    0.39    0.24675 0.6825 ] 

['V20' 'S12' 'V20' 'S16']
[0.4, 0.55, 0.4, 0.7]
[0.52  0.715 0.188 0.91 ]
[0.25 0.   0.25 0.  ]
[0.39  0.715 0.141 0.91 ] 

['V20' 'S12' 'S12' 'S16']
[0.4, 0.55, 0.55, 0.7]
[0.52   0.715  0.2585 0.91  ]
[0.   0.25 0.25 0.  ]
[0.52     0.53625  0.193875 0.91    ] 

['V20' 'S12' 'S16' 'V20']
[0.4, 0.55, 0.7, 0.4]
[0.52  0.715 0.329 0.52 ]
[0.25 0.   0.   0.25]
[0

In [ ]:
# Función que devuelve una lista con las diferentes correciones en base a la coincidencia
def Correcion_por_coincidencia(lista_horario_partido):
  Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
  lista_repeticiones = []

  unique, counts = np.unique(lista_horario_partido, return_counts=True)
  dict_valores = dict(zip(unique, counts))

  for valor in lista_horario_partido:
    repeticiones = dict_valores[valor] - 1 # restamos 1 porque si está le corresponde una correción de coincidencia de -0% no del -25%
    lista_repeticiones.append(repeticiones)

  lista_correciones = np.array(Coincidencias['-%'])[np.array(lista_repeticiones)]
  return lista_correciones

In [ ]:
# Función que devuelve una lista con las diferentes correciones en base a la coincidencia

Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
lista_repeticiones = []

unique, counts = np.unique(lista_horario_partido, return_counts=True)
dict_valores = dict(zip(unique, counts))

for valor in lista_horario_partido:
  repeticiones = dict_valores[valor] - 1 # restamos 1 porque si está le corresponde una correción de coincidencia de -0% no del -25%
  lista_repeticiones.append(repeticiones)

lista_correciones = np.array(Coincidencias['-%'])[np.array(lista_repeticiones)]
lista_correciones

array([0.45, 0.45, 0.45, 0.  ])

In [ ]:
# Función que genera dataframe
def generar_dataframe(lista_partidos: list, lista_categorias: list, lista_horario_partido:list, lista_bases: list, lista_ponderaciones:list, lista_base_ponderacion:list, lista_correciones_corregida:list ):

  dict_jornada = {
      'Partido' : lista_partidos,
      'Categorías' : lista_categorias,
      'Horario' : lista_horario_partido,
      'Base(Mill.)' : lista_bases,
      'Ponderación' : lista_ponderaciones,
      'Base*Ponderación' : lista_base_ponderacion,
      'Correción Coincidencia' : lista_correciones_corregida,
  }

  df_jornada = pd.DataFrame(dict_jornada)
  Total = np.sum(lista_correciones_corregida)
  return df_jornada, Total

In [ ]:
# Problema con menor dimension de H y P
# Creación de función que convierte matriz de combinaciones en los distintos horarios
from itertools import product
import numpy as np

lista_partidos = [
    'Celta - Real Madrid',
    'Valencia - R.Sociedad',
    'Mallorca - Eibar',
    'Athletic - Barcelona',
]

Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7}

listado_horarios = list(Ponderacion.keys())

P = len(lista_partidos)
H = len(listado_horarios)

# Generar todas las combinaciones de P números del 0 a H-1
matriz = list(product(range(H), repeat=P))
matriz_restringida = [comb for comb in matriz if (0 in comb and H-1 in comb)]

# Convertir la matriz de combinaciones con restrincciones a horarios
horarios = np.array(listado_horarios)[np.array(matriz_restringida)] # lista de listas con todos los horarios posibles para la jornada

for hora in horarios:
  print('\n', hora)



 ['V20' 'V20' 'V20' 'S16']

 ['V20' 'V20' 'S12' 'S16']

 ['V20' 'V20' 'S16' 'V20']

 ['V20' 'V20' 'S16' 'S12']

 ['V20' 'V20' 'S16' 'S16']

 ['V20' 'S12' 'V20' 'S16']

 ['V20' 'S12' 'S12' 'S16']

 ['V20' 'S12' 'S16' 'V20']

 ['V20' 'S12' 'S16' 'S12']

 ['V20' 'S12' 'S16' 'S16']

 ['V20' 'S16' 'V20' 'V20']

 ['V20' 'S16' 'V20' 'S12']

 ['V20' 'S16' 'V20' 'S16']

 ['V20' 'S16' 'S12' 'V20']

 ['V20' 'S16' 'S12' 'S12']

 ['V20' 'S16' 'S12' 'S16']

 ['V20' 'S16' 'S16' 'V20']

 ['V20' 'S16' 'S16' 'S12']

 ['V20' 'S16' 'S16' 'S16']

 ['S12' 'V20' 'V20' 'S16']

 ['S12' 'V20' 'S12' 'S16']

 ['S12' 'V20' 'S16' 'V20']

 ['S12' 'V20' 'S16' 'S12']

 ['S12' 'V20' 'S16' 'S16']

 ['S12' 'S12' 'V20' 'S16']

 ['S12' 'S12' 'S16' 'V20']

 ['S12' 'S16' 'V20' 'V20']

 ['S12' 'S16' 'V20' 'S12']

 ['S12' 'S16' 'V20' 'S16']

 ['S12' 'S16' 'S12' 'V20']

 ['S12' 'S16' 'S16' 'V20']

 ['S16' 'V20' 'V20' 'V20']

 ['S16' 'V20' 'V20' 'S12']

 ['S16' 'V20' 'V20' 'S16']

 ['S16' 'V20' 'S12' 'V20']

 ['S16' 'V20' 'S12'

In [ ]:
# Problema con menor dimension de H y P
# Creación de función que convierte matriz de combinaciones en los distintos horarios
from itertools import product
import numpy as np

lista_partidos = [
    'Celta - Real Madrid',
    'Valencia - R.Sociedad',
    'Mallorca - Eibar',
    'Athletic - Barcelona',
]

Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7}

listado_horarios = list(Ponderacion.keys())

P = len(lista_partidos)
H = len(listado_horarios)

# Generar todas las combinaciones de P números del 0 a H-1
matriz = list(product(range(H), repeat=P))
matriz_restringida = [comb for comb in matriz if (0 in comb and H-1 in comb)]
# matriz_restringida

# matriz_horarios = listado_horarios[i]

solucion = 1
hora_partido = 3
# for solucion in range(len(matriz_restringida)):
#   print(matriz_restringida[solucion])
#   horarios = np.array(listado_horarios)[np.array(matriz_restringida[solucion])]
#   print(horarios)

  # for hora_partido in range(len(lista_partidos)):
  #   valor = matriz_restringida[solucion][hora_partido]
  #   print(valor)
  #   hora = listado_horarios[matriz_restringida[solucion][hora_partido]]
  #   print(hora)

horarios = np.array(listado_horarios)[np.array(matriz_restringida)]
print(horarios)


[['V20' 'V20' 'V20' 'S16']
 ['V20' 'V20' 'S12' 'S16']
 ['V20' 'V20' 'S16' 'V20']
 ['V20' 'V20' 'S16' 'S12']
 ['V20' 'V20' 'S16' 'S16']
 ['V20' 'S12' 'V20' 'S16']
 ['V20' 'S12' 'S12' 'S16']
 ['V20' 'S12' 'S16' 'V20']
 ['V20' 'S12' 'S16' 'S12']
 ['V20' 'S12' 'S16' 'S16']
 ['V20' 'S16' 'V20' 'V20']
 ['V20' 'S16' 'V20' 'S12']
 ['V20' 'S16' 'V20' 'S16']
 ['V20' 'S16' 'S12' 'V20']
 ['V20' 'S16' 'S12' 'S12']
 ['V20' 'S16' 'S12' 'S16']
 ['V20' 'S16' 'S16' 'V20']
 ['V20' 'S16' 'S16' 'S12']
 ['V20' 'S16' 'S16' 'S16']
 ['S12' 'V20' 'V20' 'S16']
 ['S12' 'V20' 'S12' 'S16']
 ['S12' 'V20' 'S16' 'V20']
 ['S12' 'V20' 'S16' 'S12']
 ['S12' 'V20' 'S16' 'S16']
 ['S12' 'S12' 'V20' 'S16']
 ['S12' 'S12' 'S16' 'V20']
 ['S12' 'S16' 'V20' 'V20']
 ['S12' 'S16' 'V20' 'S12']
 ['S12' 'S16' 'V20' 'S16']
 ['S12' 'S16' 'S12' 'V20']
 ['S12' 'S16' 'S16' 'V20']
 ['S16' 'V20' 'V20' 'V20']
 ['S16' 'V20' 'V20' 'S12']
 ['S16' 'V20' 'V20' 'S16']
 ['S16' 'V20' 'S12' 'V20']
 ['S16' 'V20' 'S12' 'S12']
 ['S16' 'V20' 'S12' 'S16']
 

### Calcular primero la solución y después mostrar Dataframe

In [ ]:
import numpy as np
import pandas as pd
from itertools import product

# Función que devuelve una lista con las diferentes correciones en base a la coincidencia
def Correcion_por_coincidencia(lista_horario_partido):
  Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
  lista_repeticiones = []

  unique, counts = np.unique(lista_horario_partido, return_counts=True)
  dict_valores = dict(zip(unique, counts))

  for valor in lista_horario_partido:
    repeticiones = dict_valores[valor] - 1 # restamos 1 porque si está le corresponde una correción de coincidencia de -0% no del -25%
    lista_repeticiones.append(repeticiones)

  lista_correciones = np.array(Coincidencias['-%'])[np.array(lista_repeticiones)]
  return lista_correciones

# Función que genera dataframe
def generar_dataframe(lista_partidos: list, lista_categorias: list, lista_horario_partido:list, lista_bases: list, lista_ponderaciones:list, lista_base_ponderacion:list, lista_correciones_corregida:list ):

  dict_jornada = {
      'Partido' : lista_partidos,
      'Categorías' : lista_categorias,
      'Horario' : lista_horario_partido,
      'Base(Mill.)' : lista_bases,
      'Ponderación' : lista_ponderaciones,
      'Base*Ponderación' : lista_base_ponderacion,
      'Correción Coincidencia' : lista_correciones_corregida,
  }

  df_jornada = pd.DataFrame(dict_jornada)
  return df_jornada

# Condiciones iniciales para un problema fijo
##############################################
lista_partidos = [
      'Celta - Real Madrid',
      'Valencia - R.Sociedad',
      'Mallorca - Eibar',
      'Athletic - Barcelona',
      'Leganés - Osasuna',
      'Villareal - Granada',
      'Alavés - Levante',
      'Espayol - Sevilla',
  ]

lista_categorias = [
      'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B'
  ]

lista_bases = [
      1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9
  ]


Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85}

# lista_partidos = [
#     'Celta - Real Madrid',
#     'Valencia - R.Sociedad',
#     'Mallorca - Eibar',
#     'Athletic - Barcelona',
# ]

# lista_categorias = ['B-A', 'B-A', 'C-C', 'B-A']

# lista_bases = [1.3, 1.3, 0.47, 1.3]

# Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7}

lista_totales = []
dict_resultados = {}

##############################################

listado_horarios = list(Ponderacion.keys())

P = len(lista_partidos)
H = len(listado_horarios)

# Generar todas las combinaciones de P números del 0 a H-1
matriz = list(product(range(H), repeat=P))
matriz_restringida = [comb for comb in matriz if (0 in comb and H-1 in comb)]

# Convertir la matriz de combinaciones con restrincciones a horarios
horarios = np.array(listado_horarios)[np.array(matriz_restringida)] # lista de listas con todos los horarios posibles para la jornada
iteracion = 0
for lista_horario_partido in horarios:
  lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]
  lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
  lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
  lista_correciones_corregida = lista_base_ponderacion*(np.ones(len(lista_correciones))-lista_correciones)
  Total = np.sum(lista_correciones_corregida)
  lista_totales.append(Total)
  iteracion +=1
  if iteracion == 1000:
    print(iteracion)
  if iteracion == 10000:
    print(iteracion)
  if iteracion == 100000:
    print(iteracion)
  if iteracion == 1000000:
    print(iteracion)
  if iteracion == 10000000:
    print(iteracion)


lista_totales = np.array(lista_totales)
dict_resultados = dict(zip(lista_totales, [tuple(h) for h in horarios]))
horario_max_audiencia = dict_resultados.get(np.max(lista_totales))
horario_max_audiencia = [str(x) for x in horario_max_audiencia]
print(f'Solución: El horario con máxima audiencia es: {horario_max_audiencia} y obtiene una audiencia de {np.max(lista_totales)}')

KeyboardInterrupt: 

# **Problema 2. Organizar los horarios de partidos de La Liga**

- Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.
- Los horarios disponibles se conocen a priori y son los siguientes:

In [ ]:
dict_equipos = {'Celta' : 'B', 'Real Madrid' : 'A', 'Valencia' : 'B', 'R.Sociedad' : 'A', 'Mallorca' : 'C', 'Eibar' : 'C',
           'Athletic' : 'B', 'Barcelona' : 'A', 'Leganés' : 'C', 'Osasuna' : 'C', 'Villareal' : 'B', 'Granada' : 'C',
           'Alavés' : 'B', 'Levante' : 'B', 'Espayol' : 'B', 'Sevilla' : 'B', 'Betis' : 'B', 'Valladolid' : 'C', 'Atlético' : 'B',
           'Getafe' : 'B'
           }
lista_equipos = list(dict_equipos.keys())
lista_equipos

['Celta',
 'Real Madrid',
 'Valencia',
 'R.Sociedad',
 'Mallorca',
 'Eibar',
 'Athletic',
 'Barcelona',
 'Leganés',
 'Osasuna',
 'Villareal',
 'Granada',
 'Alavés',
 'Levante',
 'Espayol',
 'Sevilla',
 'Betis',
 'Valladolid',
 'Atlético',
 'Getafe']

In [ ]:
import pandas as pd
Horas = [12,16,19,20]
Dias = ['V','S','D','L']

Ponderacion = {'Viernes' : {'20h' : 0.4},
             'Sábado' : {'12h' : 0.55, '16h' : 0.7, '18h' : 0.8, '20h' : 1},
             'Domingo' : {'12h' : 0.45, '16h' : 0.75, '18h' : 0.85, '20h' : 1},
             'Lunes' : {'20h' : 0.4}
             }
Ponderacion = pd.DataFrame(Ponderacion).sort_index(ascending=True)
display(Ponderacion)

# Coincidencias = {0 : 0,
#                  1 : 0.25,
#                  2 : 0.45,
#                  3 : 0.60,
#                  4 : 0.70,
#                  5 : 0.75,
#                  6 : 0.78,
#                  7 : 0.80,
#                  8 : 0.80
# }
# lista = []
# for valor in Coincidencias:
#   lista.append(Coincidencias[valor])
# print(lista) # [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]

Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}

df_Coincidencias = pd.DataFrame(Coincidencias)
df_Coincidencias.columns.name='Coincidencias'
display(df_Coincidencias)

,Viernes,Sábado,Domingo,Lunes
12h,NaN,0.55,0.45,NaN
16h,NaN,0.70,0.75,NaN
18h,NaN,0.80,0.85,NaN
20h,0.4,1.00,1.00,0.4


Coincidencias,-%
0,0.00
1,0.25
2,0.45
3,0.60
4,0.70
5,0.75
6,0.78
7,0.80
8,0.80


In [ ]:
import random

def generar_partidos (dict_equipos):
  "Función que devuelve los partidos que se van a jugar"
  p=0 # partido
  lista_partidos = [] # lista que guarda los distintos partidos de una jornada
  lista_equipos = list(dict_equipos.keys()) #lista con todos los equipos
  lista_equipos_disponibles = lista_equipos[:]

  # Voy a ir llevando un diccionario en el que guarde los partidos jugados
  for i in range (0, len(lista_equipos),2):
    equipo_1 = random.choice(lista_equipos_disponibles)
    lista_equipos_disponibles.remove(equipo_1)
    equipo_2 = random.choice(lista_equipos_disponibles)
    lista_equipos_disponibles.remove(equipo_2)
    partido = str(equipo_1) + ' - ' + str(equipo_2)
    lista_partidos.append(partido)
  return lista_partidos

generar_partidos(dict_equipos)

['Barcelona - Athletic',
 'Villareal - Sevilla',
 'Eibar - Mallorca',
 'Celta - Valencia',
 'Real Madrid - Leganés',
 'Osasuna - Alavés',
 'Granada - R.Sociedad',
 'Getafe - Betis',
 'Espayol - Levante',
 'Atlético - Valladolid']

In [ ]:
# Asociar categorias a los partidos

import random

def generar_partidos (dict_equipos):
  "Función que devuelve los partidos que se van a jugar"
  p=0 # partido
  lista_partidos = [] # lista que guarda los distintos partidos de una jornada
  lista_categorias = [] # lista que guarda las distintas categorias de los partidos de una jornada
  lista_equipos = list(dict_equipos.keys()) #lista con todos los equipos
  lista_equipos_disponibles = lista_equipos[:]

  # Voy a ir llevando un diccionario en el que guarde los partidos jugados
  for i in range (0, len(lista_equipos),2):
    equipo_1 = random.choice(lista_equipos_disponibles)
    categoria_equipo_1 = dict_equipos[str(equipo_1)]
    lista_equipos_disponibles.remove(equipo_1)
    equipo_2 = random.choice(lista_equipos_disponibles)
    categoria_equipo_2 = dict_equipos[str(equipo_2)]
    lista_equipos_disponibles.remove(equipo_2)
    partido = str(equipo_1) + ' - ' + str(equipo_2)
    categoria = categoria_equipo_1 + '-' + categoria_equipo_2
    lista_partidos.append(partido)
    lista_categorias.append(categoria)
  return lista_partidos, lista_categorias

generar_partidos(dict_equipos)

(['Betis - Granada',
  'Osasuna - Athletic',
  'Valladolid - Mallorca',
  'Villareal - Leganés',
  'Eibar - Valencia',
  'Atlético - Espayol',
  'Sevilla - Celta',
  'Alavés - Barcelona',
  'Real Madrid - Levante',
  'Getafe - R.Sociedad'],
 ['B-C', 'C-B', 'C-C', 'B-C', 'C-B', 'B-B', 'B-B', 'B-A', 'A-B', 'B-A'])

In [ ]:
def clasificar_partido(cat_1, cat_2):

  # Generamos la tabla de audiencia base
  audiencia_base = {
      'A' : {'A' : 2},
      'B' : {'A' : 1.3, 'B' : 0.9},
      'C' : {'A' : 1, 'B' : 0.75, 'C' : 0.47}
  }

  return audiencia_base.get(cat_1,{}).get(cat_2,audiencia_base.get(cat_2,{}).get(cat_1))

In [ ]:
# Crear dataframe

import random

def generar_partidos (dict_equipos):
  "Función que devuelve los partidos que se van a jugar"
  p=0 # partido
  lista_partidos = [] # lista que guarda los distintos partidos de una jornada
  lista_categorias = [] # lista que guarda las distintas categorias de los partidos de una jornada
  lista_bases = [] # lista con las bases de audiencia de cada partido en funcion de la categoría
  lista_equipos = list(dict_equipos.keys()) #lista con todos los equipos
  lista_equipos_disponibles = lista_equipos[:]

  # Voy a ir llevando un diccionario en el que guarde los partidos jugados
  for i in range (0, len(lista_equipos),2):
    equipo_1 = random.choice(lista_equipos_disponibles)
    categoria_equipo_1 = dict_equipos[str(equipo_1)]
    lista_equipos_disponibles.remove(equipo_1)
    equipo_2 = random.choice(lista_equipos_disponibles)
    categoria_equipo_2 = dict_equipos[str(equipo_2)]
    lista_equipos_disponibles.remove(equipo_2)
    partido = str(equipo_1) + ' - ' + str(equipo_2)
    categoria = categoria_equipo_1 + '-' + categoria_equipo_2
    base = clasificar_partido(categoria_equipo_1,categoria_equipo_2)
    lista_partidos.append(partido)
    lista_categorias.append(categoria)
    lista_bases.append(base)
  return lista_partidos, lista_categorias, lista_bases

lista_partidos, lista_categorias, lista_bases = generar_partidos(dict_equipos)
dict_jornada = {
    'Partido' : lista_partidos,
    'Categorías' : lista_categorias,
    'Base(Mill.)' : lista_bases
}

df_jornada = pd.DataFrame(dict_jornada)
display(df_jornada)


,Partido,Categorías,Base(Mill.)
0,Valencia - Granada,B-C,0.75
1,Barcelona - Leganés,A-C,1.00
2,Villareal - Valladolid,B-C,0.75
3,Betis - Levante,B-B,0.90
4,Celta - Getafe,B-B,0.90
5,Real Madrid - Athletic,A-B,1.30
6,Eibar - Sevilla,C-B,0.75
7,R.Sociedad - Atlético,A-B,1.30
8,Osasuna - Alavés,C-B,0.75
9,Espayol - Mallorca,B-C,0.75


In [ ]:
# Generación de Horarios (Aleatorios en primera instancia)
# formación de los distintos horarios
listado_horarios = ['V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20', 'L20']
# lista_horario_partido = []
# for partido in lista_partidos:
#   horario = random.choice(listado_horarios)
#   lista_horario_partido.append(horario)
lista_horario_partido = [random.choice(listado_horarios) for partido in lista_partidos]
lista_horario_partido

['V20', 'L20', 'D18', 'S20', 'D18', 'D20', 'S12', 'S12', 'V20', 'V20']

In [ ]:
# Añadir Horarios - Versión Beta (Aleatorios sin restrincciones) debe tenerlas

import random

def generar_partidos (dict_equipos):
  "Función que devuelve los partidos que se van a jugar"
  p=0 # partido
  lista_partidos = [] # lista que guarda los distintos partidos de una jornada
  lista_categorias = [] # lista que guarda las distintas categorias de los partidos de una jornada
  lista_bases = [] # lista con las bases de audiencia de cada partido en funcion de la categoría
  lista_equipos = list(dict_equipos.keys()) #lista con todos los equipos
  lista_equipos_disponibles = lista_equipos[:]

  # Voy a ir llevando un diccionario en el que guarde los partidos jugados
  for i in range (0, len(lista_equipos),2):
    equipo_1 = random.choice(lista_equipos_disponibles)
    categoria_equipo_1 = dict_equipos[str(equipo_1)]
    lista_equipos_disponibles.remove(equipo_1)
    equipo_2 = random.choice(lista_equipos_disponibles)
    categoria_equipo_2 = dict_equipos[str(equipo_2)]
    lista_equipos_disponibles.remove(equipo_2)
    partido = str(equipo_1) + ' - ' + str(equipo_2)
    categoria = categoria_equipo_1 + '-' + categoria_equipo_2
    base = clasificar_partido(categoria_equipo_1,categoria_equipo_2)
    lista_partidos.append(partido)
    lista_categorias.append(categoria)
    lista_bases.append(base)

  listado_horarios = ['V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20', 'L20']
  lista_horario_partido = [random.choice(listado_horarios) for partido in lista_partidos]

  return lista_partidos, lista_categorias, lista_bases, lista_horario_partido

lista_partidos, lista_categorias, lista_bases, lista_horario_partido = generar_partidos(dict_equipos)
dict_jornada = {
    'Partido' : lista_partidos,
    'Categorías' : lista_categorias,
    'Horario' : lista_horario_partido,
    'Base(Mill.)' : lista_bases
}

df_jornada = pd.DataFrame(dict_jornada)
display(df_jornada)

,Partido,Categorías,Horario,Base(Mill.)
0,Valencia - Athletic,B-B,D12,0.90
1,Mallorca - Celta,C-B,S18,0.75
2,Eibar - R.Sociedad,C-A,D16,1.00
3,Granada - Osasuna,C-C,V20,0.47
4,Betis - Leganés,B-C,S16,0.75
5,Levante - Valladolid,B-C,S12,0.75
6,Getafe - Espayol,B-B,D18,0.90
7,Sevilla - Real Madrid,B-A,D18,1.30
8,Barcelona - Villareal,A-B,S18,1.30
9,Alavés - Atlético,B-B,S20,0.90


In [ ]:
Ponderacion = {'Viernes' : {'20h' : 0.4},
             'Sábado' : {'12h' : 0.55, '16h' : 0.7, '18h' : 0.8, '20h' : 1},
             'Domingo' : {'12h' : 0.45, '16h' : 0.75, '18h' : 0.85, '20h' : 1},
             'Lunes' : {'20h' : 0.4}
             }
Ponderacion = pd.DataFrame(Ponderacion).sort_index(ascending=True)
display(Ponderacion)

Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8, 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85, 'D20' : 1, 'L20' : 0.4}
Ponderacion

,Viernes,Sábado,Domingo,Lunes
12h,NaN,0.55,0.45,NaN
16h,NaN,0.70,0.75,NaN
18h,NaN,0.80,0.85,NaN
20h,0.4,1.00,1.00,0.4


{'V20': 0.4,
 'S12': 0.55,
 'S16': 0.7,
 'S18': 0.8,
 'S20': 1,
 'D12': 0.45,
 'D16': 0.75,
 'D18': 0.85,
 'D20': 1,
 'L20': 0.4}

In [ ]:
def Correcion_por_coincidencia(lista_horario_partido): # devuelve una lista con las diferentes correciones en base a la coincidencia
  Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
  lista_correciones = []

  for hora in lista_horario_partido:
    contador = lista_horario_partido.count(hora) - 1
    lista_correciones.append(Coincidencias['-%'][contador])
  return lista_correciones

Correcion_por_coincidencia(lista_horario_partido)


[0, 0, 0, 0, 0, 0.25, 0.25, 0, 0, 0]

In [ ]:
# Añadir Horarios - Versión Beta (Aleatorios sin restrincciones) debe tenerlas

import random
import numpy as np

def generar_partidos (dict_equipos):
  "Función que devuelve los partidos que se van a jugar"
  lista_partidos = [] # lista que guarda los distintos partidos de una jornada
  lista_categorias = [] # lista que guarda las distintas categorias de los partidos de una jornada
  lista_bases = [] # lista con las bases de audiencia de cada partido en funcion de la categoría
  lista_equipos = list(dict_equipos.keys()) #lista con todos los equipos
  lista_equipos_disponibles = lista_equipos[:]

  # Voy a ir llevando un diccionario en el que guarde los partidos jugados
  for i in range (0, len(lista_equipos),2):
    equipo_1 = random.choice(lista_equipos_disponibles)
    categoria_equipo_1 = dict_equipos[str(equipo_1)]
    lista_equipos_disponibles.remove(equipo_1)
    equipo_2 = random.choice(lista_equipos_disponibles)
    categoria_equipo_2 = dict_equipos[str(equipo_2)]
    lista_equipos_disponibles.remove(equipo_2)
    partido = str(equipo_1) + ' - ' + str(equipo_2)
    categoria = categoria_equipo_1 + '-' + categoria_equipo_2
    base = clasificar_partido(categoria_equipo_1,categoria_equipo_2)
    lista_partidos.append(partido)
    lista_categorias.append(categoria)
    lista_bases.append(base)

  Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                 'D20' : 1, 'L20' : 0.4}
  listado_horarios = list(Ponderacion.keys())
  lista_horario_partido = [random.choice(listado_horarios) for partido in lista_partidos]
  lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]

  return lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones

In [ ]:
lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones = generar_partidos(dict_equipos)
lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)

dict_jornada = {
    'Partido' : lista_partidos,
    'Categorías' : lista_categorias,
    'Horario' : lista_horario_partido,
    'Base(Mill.)' : lista_bases,
    'Ponderación' : lista_ponderaciones,
    'Base*Ponderación' : lista_base_ponderacion,
    'Correción Coincidencia' : lista_correciones_corregida,
}

df_jornada = pd.DataFrame(dict_jornada)
display(df_jornada)
Total = np.sum(lista_correciones_corregida)
print('\n Total: ',Total)

,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Alavés - Getafe,B-B,S16,0.90,0.70,0.6300,0.6300
1,Mallorca - Levante,C-B,D16,0.75,0.75,0.5625,0.5625
2,Real Madrid - Espayol,A-B,L20,1.30,0.40,0.5200,0.5200
3,Athletic - Valladolid,B-C,D20,0.75,1.00,0.7500,0.5625
4,R.Sociedad - Villareal,A-B,D20,1.30,1.00,1.3000,0.9750
5,Celta - Barcelona,B-A,S20,1.30,1.00,1.3000,1.3000
6,Leganés - Eibar,C-C,S18,0.47,0.80,0.3760,0.3760
7,Osasuna - Granada,C-C,V20,0.47,0.40,0.1880,0.1410
8,Atlético - Betis,B-B,D18,0.90,0.85,0.7650,0.7650
9,Sevilla - Valencia,B-B,V20,0.90,0.40,0.3600,0.2700



 Total:  6.102


In [ ]:
# Añadir Horarios - Versión Beta (Aleatorios sin restrincciones) debe tenerlas

import random
import numpy as np

def generar_partidos (dict_equipos):
  "Función que devuelve los partidos que se van a jugar"
  lista_partidos = [] # lista que guarda los distintos partidos de una jornada
  lista_categorias = [] # lista que guarda las distintas categorias de los partidos de una jornada
  lista_bases = [] # lista con las bases de audiencia de cada partido en funcion de la categoría
  lista_equipos = list(dict_equipos.keys()) #lista con todos los equipos
  lista_equipos_disponibles = lista_equipos[:]

  # Voy a ir llevando un diccionario en el que guarde los partidos jugados
  for i in range (0, len(lista_equipos),2):
    equipo_1 = random.choice(lista_equipos_disponibles)
    categoria_equipo_1 = dict_equipos[str(equipo_1)]
    lista_equipos_disponibles.remove(equipo_1)
    equipo_2 = random.choice(lista_equipos_disponibles)
    categoria_equipo_2 = dict_equipos[str(equipo_2)]
    lista_equipos_disponibles.remove(equipo_2)
    partido = str(equipo_1) + ' - ' + str(equipo_2)
    categoria = categoria_equipo_1 + '-' + categoria_equipo_2
    base = clasificar_partido(categoria_equipo_1,categoria_equipo_2)
    lista_partidos.append(partido)
    lista_categorias.append(categoria)
    lista_bases.append(base)

  Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                 'D20' : 1, 'L20' : 0.4}
  listado_horarios = list(Ponderacion.keys())
  lista_horario_partido = [random.choice(listado_horarios) for partido in lista_partidos]
  lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]

  return lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones

lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones = generar_partidos(dict_equipos)
lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)

dict_jornada = {
    'Partido' : lista_partidos,
    'Categorías' : lista_categorias,
    'Horario' : lista_horario_partido,
    'Base(Mill.)' : lista_bases,
    'Ponderación' : lista_ponderaciones,
    'Base*Ponderación' : lista_base_ponderacion,
    'Correción Coincidencia' : lista_correciones_corregida,
}

df_jornada = pd.DataFrame(dict_jornada)
display(df_jornada)
Total = np.sum(lista_correciones_corregida)
print('\n Total: ',Total)

In [ ]:
# Tenemos que ir variando las horas de todos los partidos probando que vayan todos desde el Viernes hasta el Lunes
lista_horario_partido = [random.choice(listado_horarios) for partido in lista_partidos]
Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                'D20' : 1, 'L20' : 0.4}
listado_horarios = list(Ponderacion.keys()) # Existen 10 tramos horarios distintos y tenemos 10 partidos
  # P A R T I D O S
# H
# O
# R
# A
# R
# I
# O
# S

# calcular total para [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] # 10 iteraciones
# calcular total para [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
# calcular total para [0, 0, 0, 0, 0, 0, 0, 0, 0, 9]

# calcular total para [0, 0, 0, 0, 0, 0, 0, 0, 1, 0] # 10 iteraciones
# calcular total para [0, 0, 0, 0, 0, 0, 0, 0, 2, 0] # 10 iteraciones

# calcular total para [9, 9, 9, 9, 9, 9, 9, 9, 9, 9] # 10 mil millones iteraciones


listado_horarios

['V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20', 'L20']

## Fijamos los partidos igual que los propuestos para optimizar un problema con el mismo arranque

In [ ]:
import random
import numpy as np

def generar_partidos (dict_equipos):
  "Función que devuelve los partidos que se van a jugar"
  lista_partidos = [] # lista que guarda los distintos partidos de una jornada
  lista_categorias = [] # lista que guarda las distintas categorias de los partidos de una jornada
  lista_bases = [] # lista con las bases de audiencia de cada partido en funcion de la categoría
  lista_equipos = list(dict_equipos.keys()) #lista con todos los equipos
  lista_equipos_disponibles = lista_equipos[:]

  lista_partidos = [
      'Celta - Real Madrid',
      'Valencia - R.Sociedad',
      'Mallorca - Eibar',
      'Athletic - Barcelona',
      'Leganés - Osasuna',
      'Villareal - Granada',
      'Alavés - Levante',
      'Espayol - Sevilla',
      'Betis - Valladolid',
      'Atlético - Getafe',
  ]

  lista_categorias = [
      'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B', 'B-C', 'B-B'
  ]

  lista_bases = [
      1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75, 0.9
  ]


  # Voy a ir llevando un diccionario en el que guarde los partidos jugados
  # for i in range (0, len(lista_equipos),2):
    # equipo_1 = random.choice(lista_equipos_disponibles)
    # categoria_equipo_1 = dict_equipos[str(equipo_1)]
    # lista_equipos_disponibles.remove(equipo_1)
    # equipo_2 = random.choice(lista_equipos_disponibles)
    # categoria_equipo_2 = dict_equipos[str(equipo_2)]
    # lista_equipos_disponibles.remove(equipo_2)
    # partido = str(equipo_1) + ' - ' + str(equipo_2)
    # categoria = categoria_equipo_1 + '-' + categoria_equipo_2
    # base = clasificar_partido(categoria_equipo_1,categoria_equipo_2)
    # lista_partidos.append(partido)
    # lista_categorias.append(categoria)
    # lista_bases.append(base)

  Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                 'D20' : 1, 'L20' : 0.4}
  listado_horarios = list(Ponderacion.keys())
  # lista_horario_partido = [random.choice(listado_horarios) for partido in lista_partidos]

  lista_horario_partido = [
      'V20', 'S12', 'S16', 'S18', 'S20', 'D16', 'D16', 'D18', 'D20', 'L20'
  ]

  lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]

  return lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones

lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones = generar_partidos(dict_equipos)
lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)

dict_jornada = {
    'Partido' : lista_partidos,
    'Categorías' : lista_categorias,
    'Horario' : lista_horario_partido,
    'Base(Mill.)' : lista_bases,
    'Ponderación' : lista_ponderaciones,
    'Base*Ponderación' : lista_base_ponderacion,
    'Correción Coincidencia' : lista_correciones_corregida,
}

df_jornada = pd.DataFrame(dict_jornada)
display(df_jornada)
Total = np.sum(lista_correciones_corregida)
print('\n Total: ',Total)

,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,V20,1.30,0.40,0.5200,0.520000
1,Valencia - R.Sociedad,B-A,S12,1.30,0.55,0.7150,0.715000
2,Mallorca - Eibar,C-C,S16,0.47,0.70,0.3290,0.329000
3,Athletic - Barcelona,B-A,S18,1.30,0.80,1.0400,1.040000
4,Leganés - Osasuna,C-C,S20,0.47,1.00,0.4700,0.470000
5,Villareal - Granada,B-C,D16,0.75,0.75,0.5625,0.421875
6,Alavés - Levante,B-B,D16,0.90,0.75,0.6750,0.506250
7,Espayol - Sevilla,B-B,D18,0.90,0.85,0.7650,0.765000
8,Betis - Valladolid,B-C,D20,0.75,1.00,0.7500,0.750000
9,Atlético - Getafe,B-B,L20,0.90,0.40,0.3600,0.360000



 Total:  5.877125


## Optimización de horarios para el caso fijo por fuerza bruta

In [ ]:
import random
import numpy as np
import pandas as pd

dict_equipos = {'Celta' : 'B', 'Real Madrid' : 'A', 'Valencia' : 'B', 'R.Sociedad' : 'A', 'Mallorca' : 'C', 'Eibar' : 'C',
           'Athletic' : 'B', 'Barcelona' : 'A', 'Leganés' : 'C', 'Osasuna' : 'C', 'Villareal' : 'B', 'Granada' : 'C',
           'Alavés' : 'B', 'Levante' : 'B', 'Espayol' : 'B', 'Sevilla' : 'B', 'Betis' : 'B', 'Valladolid' : 'C', 'Atlético' : 'B',
           'Getafe' : 'B'
           }

def Correcion_por_coincidencia(lista_horario_partido): # devuelve una lista con las diferentes correciones en base a la coincidencia
  Coincidencias = {'-%' : [0, 0.25, 0.45, 0.6, 0.7, 0.75, 0.78, 0.8, 0.8]}
  lista_correciones = []

  for hora in lista_horario_partido:
    contador = lista_horario_partido.count(hora) - 1
    lista_correciones.append(Coincidencias['-%'][contador])
  return lista_correciones

#Correcion_por_coincidencia(lista_horario_partido)

def generar_partidos (dict_equipos):
  "Función que devuelve los partidos que se van a jugar"
  lista_partidos = [] # lista que guarda los distintos partidos de una jornada
  lista_categorias = [] # lista que guarda las distintas categorias de los partidos de una jornada
  lista_bases = [] # lista con las bases de audiencia de cada partido en funcion de la categoría
  lista_equipos = list(dict_equipos.keys()) #lista con todos los equipos
  lista_equipos_disponibles = lista_equipos[:]

  lista_partidos = [
      'Celta - Real Madrid',
      'Valencia - R.Sociedad',
      'Mallorca - Eibar',
      'Athletic - Barcelona',
      'Leganés - Osasuna',
      'Villareal - Granada',
      'Alavés - Levante',
      'Espayol - Sevilla',
      'Betis - Valladolid',
      'Atlético - Getafe',
  ]

  lista_categorias = [
      'B-A', 'B-A', 'C-C', 'B-A', 'C-C', 'B-C', 'B-B', 'B-B', 'B-C', 'B-B'
  ]

  lista_bases = [
      1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75, 0.9
  ]


  Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                 'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                 'D20' : 1, 'L20' : 0.4}

  listado_horarios = list(Ponderacion.keys())
  # lista_horario_partido = [random.choice(listado_horarios) for partido in lista_partidos]

  lista_horario_partido = [
      'V20', 'S12', 'S16', 'S18', 'S20', 'D16', 'D16', 'D18', 'D20', 'L20'
  ]

  lista_ponderaciones = [Ponderacion[lista_horario_partido[i]] for i in range(len(lista_horario_partido))]

  return lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones

lista_partidos, lista_categorias, lista_bases, lista_horario_partido, lista_ponderaciones = generar_partidos(dict_equipos)
lista_base_ponderacion = np.array(lista_bases)*np.array(lista_ponderaciones)
lista_correciones = np.array(Correcion_por_coincidencia(lista_horario_partido))
lista_correciones_corregida = lista_base_ponderacion*(1-lista_correciones)

dict_jornada = {
    'Partido' : lista_partidos,
    'Categorías' : lista_categorias,
    'Horario' : lista_horario_partido,
    'Base(Mill.)' : lista_bases,
    'Ponderación' : lista_ponderaciones,
    'Base*Ponderación' : lista_base_ponderacion,
    'Correción Coincidencia' : lista_correciones_corregida,
}

df_jornada = pd.DataFrame(dict_jornada)
display(df_jornada)
Total = np.sum(lista_correciones_corregida)
print('\n Total: ',Total)

,Partido,Categorías,Horario,Base(Mill.),Ponderación,Base*Ponderación,Correción Coincidencia
0,Celta - Real Madrid,B-A,V20,1.30,0.40,0.5200,0.520000
1,Valencia - R.Sociedad,B-A,S12,1.30,0.55,0.7150,0.715000
2,Mallorca - Eibar,C-C,S16,0.47,0.70,0.3290,0.329000
3,Athletic - Barcelona,B-A,S18,1.30,0.80,1.0400,1.040000
4,Leganés - Osasuna,C-C,S20,0.47,1.00,0.4700,0.470000
5,Villareal - Granada,B-C,D16,0.75,0.75,0.5625,0.421875
6,Alavés - Levante,B-B,D16,0.90,0.75,0.6750,0.506250
7,Espayol - Sevilla,B-B,D18,0.90,0.85,0.7650,0.765000
8,Betis - Valladolid,B-C,D20,0.75,1.00,0.7500,0.750000
9,Atlético - Getafe,B-B,L20,0.90,0.40,0.3600,0.360000



 Total:  5.877125


Tenemos que definir una función que varíe los horarios y recalcule el total, quedándonos con el máximo total encontrado

In [ ]:
Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                'D20' : 1, 'L20' : 0.4}

listado_horarios = list(Ponderacion.keys())
listado_horarios[0]

print(len(lista_partidos), len(listado_horarios))
# fila indica horario
# columna indica el horario del index partido
# [2, 0, 0, 0, 0, 0, 0, 0, 0, 0] -> Primer partido es el S16

lista_index_horario_partido = [0 for _ in range(len(lista_partidos))]
for j in range(len(lista_partidos)): # j: 0 → 9
  if j != 0 and i = 0:

  for i in range(len(listado_horarios)): # i: 0 → 9
    lista_index_horario_partido[j] = i
    print(lista_index_horario_partido)


SyntaxError: invalid syntax (<ipython-input-6-2434276614>, line 15)

In [ ]:
Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                'D20' : 1, 'L20' : 0.4}

listado_horarios = list(Ponderacion.keys())
listado_horarios[0]

# fila indica horario
# columna indica el horario del index partido
# [2, 0, 0, 0, 0, 0, 0, 0, 0, 0] -> Primer partido es el S16

lista_index_horario_partido = [0 for _ in range(len(lista_partidos))]

j=0
while j<10:
  for i in range(len(listado_horarios)+1): # i: 0 → 9
    if i<10:
      lista_index_horario_partido[j] = i
      print(lista_index_horario_partido)
    else:
      i = 1
      j += 1

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[4, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[6, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[7, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[8, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[9, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[9, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[9, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[9, 2, 0, 0, 0, 0, 0, 0, 0, 0]
[9, 3, 0, 0, 0, 0, 0, 0, 0, 0]
[9, 4, 0, 0, 0, 0, 0, 0, 0, 0]
[9, 5, 0, 0, 0, 0, 0, 0, 0, 0]
[9, 6, 0, 0, 0, 0, 0, 0, 0, 0]
[9, 7, 0, 0, 0, 0, 0, 0, 0, 0]
[9, 8, 0, 0, 0, 0, 0, 0, 0, 0]
[9, 9, 0, 0, 0, 0, 0, 0, 0, 0]
[9, 9, 0, 0, 0, 0, 0, 0, 0, 0]
[9, 9, 1, 0, 0, 0, 0, 0, 0, 0]
[9, 9, 2, 0, 0, 0, 0, 0, 0, 0]
[9, 9, 3, 0, 0, 0, 0, 0, 0, 0]
[9, 9, 4, 0, 0, 0, 0, 0, 0, 0]
[9, 9, 5, 0, 0, 0, 0, 0, 0, 0]
[9, 9, 6, 0, 0, 0, 0, 0, 0, 0]
[9, 9, 7, 0, 0, 0, 0, 0, 0, 0]
[9, 9, 8, 0, 0, 0, 0, 0, 0, 0]
[9, 9, 9, 0, 0, 0, 0, 0, 0, 0]
[9, 9, 9, 0, 0, 0, 0, 0, 0, 0]
[9, 9, 9, 1, 0, 0, 0, 0, 0, 0]
[9, 9, 9

In [ ]:
Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                'D20' : 1, 'L20' : 0.4}

listado_horarios = list(Ponderacion.keys())
listado_horarios[0]

# fila indica horario
# columna indica el horario del index partido
# [2, 0, 0, 0, 0, 0, 0, 0, 0, 0] -> Primer partido es el S16

lista_index_horario_partido = [0 for _ in range(len(lista_partidos))]

# for i in range(len(listado_horarios)): # i: 0 → 9
#   lista_index_horario_partido[0] = i
#   print(lista_index_horario_partido)

# for i in range(1,len(listado_horarios)): # i: 0 → 9
#   lista_index_horario_partido[1] = i
#   print(lista_index_horario_partido)

# while lista_index_horario_partido[len(lista_index_horario_partido)-1] < 9

j=0
contador=0
for j in range(len(lista_partidos)):
  for i in range(1,len(listado_horarios)+1): # i: 1 → 10
    if i == len(listado_horarios): #10
      k = j
      while k > -1: # poner a cero todos los valores anteriores y sumar 1 a la siguiente posición
        lista_index_horario_partido[k] = 0
        k -=1
      j +=1
      lista_index_horario_partido[j] = lista_index_horario_partido[j] + 1
      print(lista_index_horario_partido)
    else:
      lista_index_horario_partido[0] = i
      print(lista_index_horario_partido)

  #   lista_index_horario_partido[j] = 0
  #   lista_index_horario_partido[j+1] = 1
  # print(lista_index_horario_partido)


NameError: name 'lista_partidos' is not defined

In [ ]:
Ponderacion = {'V20': 0.4, 'S12' : 0.55, 'S16' : 0.7, 'S18' : 0.8,
                'S20' : 1, 'D12' : 0.45, 'D16' : 0.75, 'D18' : 0.85,
                'D20' : 1, 'L20' : 0.4}

listado_horarios = list(Ponderacion.keys())
listado_horarios[0]

# fila indica horario
# columna indica el horario del index partido
# [2, 0, 0, 0, 0, 0, 0, 0, 0, 0] -> Primer partido es el S16

lista_index_horario_partido = [0 for _ in range(len(lista_partidos))]

j=0
for j in range(len(lista_partidos)):
  # while lista_index_horario_partido[j] < 9:
  for i in range(1,len(listado_horarios)+1):
    if i == len(listado_horarios):

      k = j
      while k > -1: # lo primero es poner a cero todos los valores anteriores
        lista_index_horario_partido[k] = 0
        k -=1


    else:
      lista_index_horario_partido[j] = i
      print(lista_index_horario_partido)





# lista_index_horario_partido = [0 for _ in range(len(lista_partidos))]
# j=0
# for j in range(len(lista_partidos)):
#   for i in range(1,len(listado_horarios)+1): # i: 1 → 10
#     if i == len(listado_horarios): # si hay desbordamiento
#       k = j
#       while k > -1: # lo primero es poner a cero todos los valores anteriores
#         lista_index_horario_partido[k] = 0
#         k -=1
#       # lo segundo es sumar 1 al siguiente valor
#       if lista_index_horario_partido[j] == 9:
#         pass
#       else:
#         lista_index_horario_partido[j] = lista_index_horario_partido[j] + 1
#       print(lista_index_horario_partido)
#     else:
#       lista_index_horario_partido[0] = i
#       print(lista_index_horario_partido)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 5]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 6]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 7]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 8]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 9]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 9]
[2, 0, 0, 0, 0, 0, 0, 0, 0, 9]
[3, 0, 0, 0, 0, 0, 0, 0, 0, 9]
[4, 0, 0, 0, 0, 0, 0, 0, 0, 9]
[5, 0, 0, 0, 0, 0, 0, 0, 0, 9]
[6, 0, 0, 0, 0, 0, 0, 0, 0, 9]
[7, 0, 0, 0, 0, 0, 0, 0, 0, 9]
[8, 0, 0, 0, 0, 0, 0, 0, 0, 9]
[9, 0, 0, 0, 0, 0, 0, 0, 0, 9]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 9]
[0, 2, 0, 0, 0, 0, 0, 0, 0, 9]
[0, 3, 0, 0, 0, 0, 0, 0, 0, 9]
[0, 4, 0, 0, 0, 0, 0, 0, 0, 9]
[0, 5, 0, 0, 0, 0, 0, 0, 0, 9]
[0, 6, 0, 0, 0, 0, 0, 0, 0, 9]
[0, 7, 0, 0, 0, 0, 0, 0, 0, 9]
[0, 8, 0, 0, 0, 0, 0, 0, 0, 9]
[0, 9, 0, 0, 0, 0, 0, 0, 0, 9]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 9]
[0, 0, 2, 0, 0, 0, 0, 0, 0, 9]
[0, 0, 3, 0, 0, 0, 0, 0, 0, 9]
[0, 0, 4, 0, 0, 0, 0, 0, 0, 9]
[0, 0, 5, 0, 0, 0, 0, 0, 0, 9]
[0, 0, 6

In [ ]:
# Crea lista que va del 1-9 en cada columna
lista_index_horario_partido = [0 for _ in range(len(lista_partidos))]

j=0
for j in range(len(lista_partidos)):
  # while lista_index_horario_partido[j] < 9:
  for i in range(1,len(listado_horarios)+1):
    if i == len(listado_horarios):

      k = j
      while k > -1: # lo primero es poner a cero todos los valores anteriores
        lista_index_horario_partido[k] = 0
        k -=1


    else:
      lista_index_horario_partido[j] = i
      print(lista_index_horario_partido)

In [ ]:
from itertools import product


# Esto genera todas las combinaciones de 4 números del 0 al 9

matriz = list(product(range(len(listado_horarios)), repeat=8))
%time
matriz
# Mostrar las combinaciones
# for combinacion in matriz:
#     print(combinacion)